**IMPORTAR LIBRERIAS**

---

In [4]:
# Librerias de manejo de datos
import pandas as pd

In [5]:
# Importar conexión
from db_connection import get_connection

# Conectarse a la base de datos
conexion = get_connection()
cursor = conexion.cursor()


✅ Conexión exitosa a la base de datos


In [6]:
# Importar librerías para la conexión a la base de datos con SQLAlchemy
from db_connection import get_engine

engine = get_engine()

In [7]:
# 📌 Obtener todas las tablas
cursor.execute("SHOW TABLES;")
tablas = [fila[0] for fila in cursor.fetchall()]
print(f"Total de tablas encontradas: {len(tablas)}")


Total de tablas encontradas: 50


---

**FUNCIONES**

---

In [8]:
def borrar_lista_tablas(tablas_a_borrar):
    """
    Borra las tablas de la base de datos que se pasan como argumento.
    """
    for tabla in tablas_a_borrar:
        try:
            cursor.execute(f"DROP TABLE `{tabla}`;")
            tablas.remove(tabla)
            print(f"✅ Tabla eliminada: {tabla}")
        except Exception as e:
            print(f"❌ Error al eliminar {tabla}: {e}")
    
    # Confirmar cambios
    conexion.commit()

------

**FILTROS PARA SEGREGAR Y ELIMINAR LAS TABLAS QUE NO SON ÚTILES**


---

*PRIMER FILTRO: Tablas con menos de 2 registros*

In [9]:
# Escoger tablas a eliminar
tablas_a_borrar = []

for tabla in tablas:
    try:
        cursor.execute(f"SELECT COUNT(*) FROM `{tabla}`;")
        total = cursor.fetchone()[0]
        if total <= 1:
            tablas_a_borrar.append(tabla)
    except Exception as e:
        print(f"❌ Error evaluando tabla {tabla}: {e}")

print(f"\nTablas candidatas para eliminación ({len(tablas_a_borrar)}):")
print(tablas_a_borrar)



Tablas candidatas para eliminación (0):
[]


In [10]:
# ⚠️ Solo si tienes respaldo y estás en entorno seguro
# ELIMINAR TABLAS CANDIDATAS

# Desactivar restricciones
cursor.execute("SET FOREIGN_KEY_CHECKS=0;")

borrar_lista_tablas(tablas_a_borrar)

# Volver a activar restricciones
cursor.execute("SET FOREIGN_KEY_CHECKS=1;")



---

*SEGUNDO FILTRO: Tablas posiblemente innecesarias por el nombre y breve análisis*

In [11]:
# Imprimir los nombres de las tablas restantes
for tabla in tablas:
    print(tabla)


accounting_account_balances
accounting_accounts
accounting_movements
accounting_voucher_items
accounting_voucher_types
accounting_vouchers
billing_numberings
contact_relationships
contact_statements
contacts
contract_salary_history
costs_and_expenses
costs_and_expenses_categories
coupons
document_items
documents
ecommerce_legal_info
ecommerce_purchase_orders
employee_contracts
employee_positions
employees
fixed_asset_depreciations
fixed_assets
fixed_assets_groups
headquarter_warehouses
headquarters
inventory_adjustments
inventory_groups
item_balance
item_categories
item_depreciations
item_kardex
item_subcategories
item_variations
items
payment_conditions
payments
payroll_deductions
payroll_details
payroll_incomes
payroll_providers
payrolls
price_lists
retention_concepts
retentions
retentions_applied
retentions_certificates
taxes
warehouse_transfer_logs
warehouses


*ANÁLISIS Y CONCLUSIONES DE LA INFORMACIÓN OBTENIDA*

**Tablas clave para análisis de gestión empresarial (💡 Mantener)**

Estas son probablemente las más útiles para análisis de KPIs, dashboards o visualización de negocio:

*Categoría	//  Tablas clave*

📦 Inventario	= items, item_balance, item_kardex, warehouses, inventory_adjustments, item_categories, item_subcategories, item_variations

💼 Ventas / Documentos	= documents, document_items, contacts (clientes/proveedores)

🧾 Contabilidad	= accounting_vouchers, accounting_voucher_items, accounting_accounts, accounting_movements, retentions, retention_concepts, retentions_applied, accounting_account_balances

👥 Empleados / Nómina	= employees, employee_contracts, payrolls, payroll_details, payroll_incomes, payroll_deductions

🛒 Compras	= ecommerce_purchase_orders

📊 Precios	= price_lists

🏪 Organización	= headquarters, user_headquarters, roles, user_roles, company_areas

🧾 Impuestos	= taxes

🏭 Activos fijos	= fixed_assets, fixed_asset_depreciations, fixed_assets_groups


**Relevancia baja o dependiente del contexto (🤔 Revisar)**

*Estas podrían ser útiles dependiendo del foco del proyecto (clientes, pedidos, legal, e-commerce):*

contact_relationships, contact_accounts, contact_statements

coupons, coupon_groups (si hay ecommerce)

employee_positions, contract_salary_history

plan_electronic_documents, plan_system_controller

ecommerce_legal_info

costs_and_expenses, costs_and_expenses_categories

radian_documents (puede ser de facturación electrónica)

dining_tables (¿restaurante? puede ser irrelevante)

system_counters, system_restrictions

In [12]:
# Lista de tablas candidatas a eliminar
tablas_candidatas = [
    "api_access_tokens",
    "electronic_payroll_test_set",
    "migrations",
    "oauth_clients",
    "paynilo",
    "role_permissions",
    "template_versions",
    "templates",
    "user_data",
    "values_x_item",
    "warehouse_transfer_logs",
    "billing_numberings",
    "custom_fields",
    "configurations",
    "system_counters",
    "system_restrictions"
]

---

*TERCER FILTRO: tablas a eliminar según sus columnas y número de filas*

In [27]:
# Script para limpiar y aplanar columnas JSON en 'employees' (VERSIÓN DIRECTA CON FK_CHECKS DESHABILITADO)

import pandas as pd
import json
import numpy as np
from db_connection import get_engine, get_connection

print("\n--- Bloque 6 (VERSIÓN DIRECTA): Limpieza de JSONs en 'employees' con Pandas ---")

conexion = None
cursor = None
try:
    print("Reestableciendo conexión a la base de datos para el bloque de 'employees'...")
    conexion = get_connection()
    cursor = conexion.cursor()
    engine = get_engine()
    print("✅ Conexión y cursor reestablecidos exitosamente para 'employees'.")
    print("✅ Engine de base de datos obtenido.")

    # --- PASO CRÍTICO: Deshabilitar temporalmente las verificaciones de claves foráneas ---
    # Esto permite realizar el ALTER TABLE sin problemas de IDs huérfanos.
    print("\nDeshabilitando temporalmente las verificaciones de claves foráneas...")
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")
    print("✅ Verificaciones de claves foráneas deshabilitadas.")

    # --- PASO 1: Modificar la estructura de la tabla 'employees' ---
    # Convertir las columnas JSON a VARCHAR(255) NULL
    print("\nModificando las columnas JSON a VARCHAR(255) NULL...")
    alter_table_sql = """
        ALTER TABLE employees
        MODIFY COLUMN identity_document_type VARCHAR(255) NULL,
        MODIFY COLUMN country VARCHAR(255) NULL,
        MODIFY COLUMN region VARCHAR(255) NULL,
        MODIFY COLUMN city VARCHAR(255) NULL,
        MODIFY COLUMN emergency_contact VARCHAR(255) NULL,
        MODIFY COLUMN payment_method VARCHAR(255) NULL,
        MODIFY COLUMN bank VARCHAR(255) NULL,
        MODIFY COLUMN bank_account_type VARCHAR(255) NULL;
    """
    try:
        cursor.execute(alter_table_sql)
        conexion.commit() # Confirmar el cambio de estructura
        print("✅ Columnas JSON modificadas a VARCHAR(255) NULL exitosamente.")
    except Exception as e:
        conexion.rollback()
        print(f"❌ Error al modificar las columnas JSON: {e}")
        raise # Re-lanza el error para detener el script

    # --- PASO CRÍTICO: Re-habilitar las verificaciones de claves foráneas ---
    # Es FUNDAMENTAL re-habilitarlas inmediatamente después.
    print("\nRe-habilitando las verificaciones de claves foráneas...")
    cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")
    print("✅ Verificaciones de claves foráneas re-habilitadas.")

    # 1. Cargar la tabla 'employees' completa en un DataFrame de Pandas
    print("\nCargando la tabla 'employees' en un DataFrame de Pandas...")
    df_employees = pd.read_sql("SELECT * FROM employees;", engine)
    print(f"DataFrame de 'employees' cargado. Registros: {len(df_employees)}")

    # Lista de columnas que eran JSON (ahora son VARCHAR en la DB)
    json_columns = [
        'identity_document_type', 'country', 'region', 'city',
        'emergency_contact', 'payment_method', 'bank', 'bank_account_type'
    ]

    # 2. Iterar sobre cada columna (ahora VARCHAR, pero con contenido JSON previo) y aplanarla
    print("\nIniciando aplanamiento de columnas (previamente JSON) en Pandas...")

    def extract_json_value(json_str):
        # Manejar casos donde la columna ya fue aplanada o es None/NaN
        if pd.isna(json_str) or not isinstance(json_str, str) or not json_str.strip():
            return None
        # Si la cadena no parece JSON (ej. ya es un valor aplanado), devolverla tal cual
        if not json_str.strip().startswith('{') and not json_str.strip().startswith('['):
            return json_str # Ya está aplanado, o es un string simple no JSON
        try:
            data = json.loads(json_str)
            if isinstance(data, dict):
                if 'description' in data and data['description'] is not None:
                    return str(data['description'])
                elif 'name' in data and data['name'] is not None:
                    return str(data['name'])
                elif 'code' in data and data['code'] is not None:
                    return str(data['code'])
            return None
        except json.JSONDecodeError:
            # Si no es un JSON válido, pero es una cadena, devolverla.
            # Esto maneja el caso donde la columna ya fue aplanada manualmente o contiene basura.
            return json_str if json_str.strip() else None
        except Exception as e:
            return None

    for col in json_columns:
        print(f"Procesando columna: '{col}'...")
        if col in df_employees.columns:
            df_employees[col] = df_employees[col].astype(str)
            df_employees[col] = df_employees[col].apply(extract_json_value)
            print(f"   Columna '{col}' aplanada a valor extraído o NULL.")
        else:
            print(f"   Advertencia: Columna '{col}' no encontrada en el DataFrame.")
    print("✅ Aplanamiento de columnas completado.")

    # 3. Manejo de valores nulos y tipos de datos para la actualización
    print("\nPreparando DataFrame para la actualización (manejo de nulos y tipos)...")

    # Volvemos a obtener la descripción de la tabla para asegurarnos de que los tipos son los nuevos (VARCHAR)
    cursor.execute("DESCRIBE `employees`;")
    db_columns_info = cursor.fetchall()
    db_column_names = [col_info[0] for col_info in db_columns_info]
    db_column_types = {col_info[0]: col_info[1] for col_info in db_columns_info}

    valid_columns_for_update = [
        col for col in df_employees.columns
        if isinstance(col, str) and col != 'id' and col in db_column_names
    ]

    for col in df_employees.columns:
        if col in valid_columns_for_update or col == 'id':
            if df_employees[col].dtype == 'object':
                df_employees[col] = df_employees[col].replace({np.nan: None, 'nan': None, '': None})
            elif pd.api.types.is_numeric_dtype(df_employees[col]) and df_employees[col].isnull().any():
                if pd.api.types.is_integer_dtype(df_employees[col]):
                     df_employees[col] = df_employees[col].astype(pd.Int64Dtype())
                df_employees[col] = df_employees[col].replace({np.nan: None})

            if pd.api.types.is_bool_dtype(df_employees[col]):
                df_employees[col] = df_employees[col].map({True: 1, False: 0, None: None, np.nan: None})

            if pd.api.types.is_string_dtype(df_employees[col]) and col in db_column_types:
                mysql_type = db_column_types[col].lower()
                if 'varchar' in mysql_type:
                    try:
                        max_len = int(mysql_type.split('(')[1].split(')')[0])
                        long_strings = df_employees[df_employees[col].astype(str).str.len() > max_len][col]
                        if not long_strings.empty:
                            print(f"   ⚠️ ADVERTENCIA: Columna '{col}' tiene strings más largos que {max_len} caracteres.")
                            print(f"     Primeros 5 valores problemáticos: {long_strings.head().tolist()}")
                    except (IndexError, ValueError):
                        pass

    print("✅ DataFrame preparado para la actualización.")

    # 4. Actualizar los registros con los datos limpios del DataFrame
    print("\n--- Actualizando registros en 'employees' con datos limpios ---")

    set_clauses = [f"`{col}` = %s" for col in valid_columns_for_update]
    update_sql = f"UPDATE `employees` SET {', '.join(set_clauses)} WHERE `id` = %s;"

    print(f"DEBUG: SQL de UPDATE generado: {update_sql[:150]}...")
    print(f"DEBUG: Columnas que se actualizarán: {valid_columns_for_update}")

    updated_count = 0
    errors_encountered = 0
    for index, row in df_employees.iterrows():
        update_values = []
        for col in valid_columns_for_update:
            val = row[col]
            if pd.isna(val) or (isinstance(val, (float, np.float64)) and np.isnan(val)):
                update_values.append(None)
            elif isinstance(val, (np.int64, np.int32)):
                update_values.append(int(val))
            elif isinstance(val, (np.bool_)):
                update_values.append(bool(val))
            else:
                update_values.append(val)

        update_values.append(row['id'])

        try:
            cursor.execute(update_sql, tuple(update_values))
            updated_count += 1
            if updated_count % 5 == 0:
                print(f"   Progreso: {updated_count}/{len(df_employees)} registros procesados...")
        except Exception as e:
            errors_encountered += 1
            print(f"\n❌❌ ERROR CRÍTICO al actualizar el registro con ID {row['id']}: {e}")
            print(f"   SQL: {update_sql}")
            print(f"   Valores problemáticos (ID={row['id']}): {tuple(update_values)}")
            raise

    if errors_encountered > 0:
        conexion.rollback()
        print(f"❌ Actualización finalizada con {errors_encountered} errores. Transacción revertida.")
    else:
        conexion.commit()
        print(f"✅ Actualización de {updated_count} registros en la tabla 'employees' completada exitosamente.")

    print("\n✅ Limpieza y aplanamiento de JSONs en 'employees' finalizada exitosamente.")

except Exception as e:
    print(f"\n❌❌ Ocurrió un error general NO CAPTURADO en el bucle principal: {e}")
    print("Por favor, revisa la conexión a la base de datos o los datos en sí.")
    try:
        if conexion and conexion.is_connected():
            conexion.rollback()
            print("Transacción revertida debido a un error.")
    except Exception as rollback_e:
        print(f"❌ Error adicional al intentar revertir la transacción: {rollback_e}")
finally:
    # Asegúrate de re-habilitar las FKs incluso si hay un error en el try-except.
    # Esto es CRÍTICO para mantener la integridad de tu base de datos.
    if cursor is not None:
        try:
            cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")
            print("✅ Verificaciones de claves foráneas re-habilitadas en el bloque 'finally'.")
        except Exception as e:
            print(f"❌ Advertencia: No se pudieron re-habilitar las FKs en finally: {e}")

    # La conexión a la base de datos se mantiene abierta según la solicitud.
    print("La conexión a la base de datos se mantiene abierta según la solicitud.")


--- Bloque 6 (VERSIÓN DIRECTA): Limpieza de JSONs en 'employees' con Pandas ---
Reestableciendo conexión a la base de datos para el bloque de 'employees'...
✅ Conexión exitosa a la base de datos
✅ Conexión y cursor reestablecidos exitosamente para 'employees'.
✅ Engine de base de datos obtenido.

Deshabilitando temporalmente las verificaciones de claves foráneas...
✅ Verificaciones de claves foráneas deshabilitadas.

Modificando las columnas JSON a VARCHAR(255) NULL...
✅ Columnas JSON modificadas a VARCHAR(255) NULL exitosamente.

Re-habilitando las verificaciones de claves foráneas...
✅ Verificaciones de claves foráneas re-habilitadas.

Cargando la tabla 'employees' en un DataFrame de Pandas...
DataFrame de 'employees' cargado. Registros: 38

Iniciando aplanamiento de columnas (previamente JSON) en Pandas...
Procesando columna: 'identity_document_type'...
   Columna 'identity_document_type' aplanada a valor extraído o NULL.
Procesando columna: 'country'...
   Columna 'country' aplana

**Tablas candidatas fuertes para eliminación (por contenido y nombre)**

Estas tablas tienen pocos registros, estructuras irrelevantes para KPIs o funciones accesorias que no impactan directamente en la gestión:

*Tabla	// Motivo principal*

api_access_tokens	Manejo técnico de tokens

electronic_payroll_test_set	Datos de prueba (test set)

migrations	Solo para migraciones del sistema

oauth_clients	Seguridad/autenticación interna

paynilo	Integración de pagos externa

role_permissions	Control de permisos internos

template_versions	Versiones de plantillas, sin datos

templates	Estructura visual, no analítica

user_data	Solo datos de área/cargo del usuario

custom_fields	Campos personalizados (metadatos)

system_counters	Estadísticas internas del sistema

system_restrictions	Restricciones de uso/licencia

values_x_item	Datos muy técnicos por item, posiblemente redundantes

warehouse_transfer_logs	Registro técnico, no necesariamente útil para KPIs

----

*CUARTO FILTRO: Investigar mas a fondo las tablas dudosas*

**⚠️ Tablas dudosas (verificar con .info() antes de decidir)**

Estas tienen algo más de contenido o potencial analítico dependiendo del negocio:

*billing_numberings:* Tiene 155 registros. Podría contener información útil para facturación si los números se asocian a documentos.

*configurations:* Solo 25 registros, pero hay que ver si son configuraciones claves que afectan reportes.



In [14]:
# Obtener el esquema de cada tabla para decidir si son candidatas a eliminación
tablas_dudosas = ['billing_numberings', 'configurations'] # Agregar mas tablas si es necesario

for tabla in tablas_dudosas:
    print(f"\n📋 Tabla: {tabla}")
    try:
        df = pd.read_sql(f"SELECT * FROM {tabla}", con=engine)
        print("▶️ .info():")
        print(df.info())
        print("▶️ Primeras filas:")
        display(df.head())
    except Exception as e:
        print(f"❌ Error al cargar {tabla}: {e}")



📋 Tabla: billing_numberings


▶️ .info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      50 non-null     int64         
 1   document_type_id        50 non-null     int64         
 2   pos_numbering           50 non-null     int64         
 3   name                    50 non-null     object        
 4   prefix                  50 non-null     object        
 5   initial_number          50 non-null     int64         
 6   current_number          50 non-null     int64         
 7   final_number            22 non-null     float64       
 8   valid_since             22 non-null     object        
 9   valid_until             22 non-null     object        
 10  validity                22 non-null     float64       
 11  preferred               50 non-null     int64         
 12  electronic_billing      50 non-null    

,id,document_type_id,pos_numbering,name,prefix,initial_number,current_number,final_number,valid_since,valid_until,...,tech_key,used,template,headquarter_id,warehouse_id,status,user_id,created_at,updated_at,contingency
0,1,11,0,Remisiones,,0,56,NaN,None,None,...,None,1,"{""id"": 1, ""name"": ""Carta"", ""template_type"": ""1""}",1,1,1,10,NaT,2024-05-24 09:54:00,0
1,2,12,0,Cotizaciones,,0,91,NaN,None,None,...,None,1,"{""id"": 3, ""name"": ""POS"", ""template_type"": ""1""}",1,1,1,10,NaT,2024-07-13 11:53:24,0
2,3,13,0,Recibos de Caja,,0,506,NaN,None,None,...,None,1,"{""id"": 1, ""name"": ""Carta"", ""template_type"": ""1""}",1,1,1,10,NaT,2024-07-17 10:58:41,0
3,4,14,0,Comprobantes de Pago,,0,113,NaN,None,None,...,None,1,"{""id"": 1, ""name"": ""Carta"", ""template_type"": ""1""}",1,1,1,0,NaT,2024-07-17 11:00:31,0
4,5,15,0,Órdenes de Compra,,0,23,NaN,None,None,...,None,1,"{""id"": 1, ""name"": ""Carta"", ""template_type"": ""1""}",1,1,1,0,NaT,2024-04-03 16:10:15,0



📋 Tabla: configurations
❌ Error al cargar configurations: (mysql.connector.errors.ProgrammingError) 1146 (42S02): Table 'novateam.configurations' doesn't exist
[SQL: SELECT * FROM configurations]
(Background on this error at: https://sqlalche.me/e/20/f405)


---

**TABLAS FINALES A ELIMINAR DESPUÉS DE TODOS LOS FILTROS Y ANÁLISIS INDIVIDUAL**

Falta escoger y seleccionar individualmente 

In [15]:
tablas_a_eliminar = [
    "api_access_tokens",             # Manejo técnico de tokens
    "electronic_payroll_test_set",   # Datos de prueba (test set)
    "migrations",                    # Solo para migraciones del sistema
    "oauth_clients",                 # Seguridad/autenticación interna
    "paynilo",                       # Integración de pagos externa
    "role_permissions",             # Control de permisos internos
    "template_versions",            # Versiones de plantillas, sin datos
    "templates",                    # Estructura visual, no analítica
    "user_data",                    # Solo datos de área/cargo del usuario
    "custom_fields",                # Campos personalizados (metadatos)
    "system_counters",              # Estadísticas internas del sistema
    "system_restrictions",          # Restricciones de uso/licencia
    "values_x_item",                # Datos técnicos por ítem
    "user_roles",
    "user_data",
    "user_headquarters",
    "term_and_conditions,",
    "severance_payments",
    "roles", 
    "role_permissions",
    "radian_events",
    "radian_documents",
    "plan_system_controller",
    "plan_electronic_documents",
    "api_access_tokens",
    "configurations",
    "company_areas",
    "consolidated_retention_certificates",
    "contact_accounts",
    "contact_relationship",
    "contact_statement",
    "contact",
    "custom_fields",
    "dining_tables",
    "ecomerce_legal_info",
    "ecomerce_purchase_orders",
    "electronic_payroll_test_set", 
    "electronic_contracts",
    "migraciones ]"     # Registro técnico, no útil para KPIs

]




In [ ]:
# ⚠️ Solo si tienes respaldo y estás en entorno seguro
# ELIMINAR TABLAS FINALES ¡OJO HAY QUE ESCOGER ANTES DE EJECUTAR!

# Desactivar restricciones !NO SE HACE ANTES DE VERIFICAR QUE NO GENERA NINGÚN DAÑO!
# cursor.execute("SET FOREIGN_KEY_CHECKS=0;")

# borrar_lista_tablas(tablas_a_eliminar)

# Volver a activar restricciones !NO SE HACE ANTES DE VERIFICAR QUE NO GENERA NINGÚN DAÑO!
#cursor.execute("SET FOREIGN_KEY_CHECKS=1;")
#

**TABLAS ASIGNADAS AL ANÁLISIS DE RH - ALEXANDRA**
A continuación, se irá explorando una a una, las tablas y su contenido para visualizar qué tan viables son para el ejercicio propuesto, se realizará limpieza de datos, se eliminarán nulls según convenga o se reemplazarán por n/a.  

employee_positions, employees, , employee_contracts, contract_salary_history, payrolls, payroll_details, payroll_incomes, payroll_deductions, payroll_providers, costs_and_expenses, costs_and_expenses_categories, headquarters, coupons. 



In [ ]:
#TABLA 1: EMPLOYEE_POSITIONS  O Posiciones de empleados. 
# Script para obtener la información de las columnas de 'employee_positions'

# Conectarse a la base de datos
conexion = get_connection()
cursor = conexion.cursor()

try:
    # Obtener la descripción de las columnas de la tabla 'employee_positions'
    cursor.execute("DESCRIBE employee_positions;")
    columnas_employee_positions = cursor.fetchall()

    print("Columnas de la tabla 'employee_positions':")
    for columna in columnas_employee_positions:
        # Cada 'columna' es una tupla con (nombre_columna, tipo_dato, ...)
        print(f"  - {columna[0]} ({columna[1]})")

except Exception as e:
    print(f"Ocurrió un error al intentar obtener las columnas de 'employee_positions': {e}")

✅ Conexión exitosa a la base de datos
Columnas de la tabla 'employee_positions':
  - id (int unsigned)
  - name (varchar(255))
  - description (varchar(500))
  - salary (double(16,2))
  - status (tinyint(1))
  - user_id (int unsigned)
  - created_at (timestamp)
  - updated_at (timestamp)


📊 Análisis Profundo de la Tabla employee_positions

🎯 *Propósito y Relevancia Estratégica en RRHH*

La tabla employee_positions es tu diccionario de cargos. Es donde cada puesto de trabajo en la empresa cobra vida, describiendo su rol y, a menudo, su valor salarial de referencia. Es fundamental para:

Estructuración Organizacional: ¿Quién hace qué?
Gestión de Compensaciones: ¿Cuánto valoramos cada rol?
Planificación de Personal: ¿Qué roles necesitamos cubrir o crear?

🔍 Columnas Clave y Consideraciones Cruciales

id: int unsigned

Identificador Único: La ✨ clave primaria ✨ de la tabla. Garantiza que cada posición es unívoca.
Limpieza: No requiere acción. Es tu núcleo de referencia.
name: varchar(255)

Nombre del Puesto: El nombre que todos reconocen (ej., "Gerente de Marketing", "Ingeniero de Software").
Limpieza: ¡Aquí hay trabajo! Busca variaciones o duplicados (ej., "Developer" vs. "Desarrollador"). La estandarización es clave para análisis coherentes. 🧼 Si hay nulos o vacíos, considera rellenar con "Sin Nombre de Posición".
description: varchar(500)

Detalle del Rol: Un texto que expande las responsabilidades de la posición.
Limpieza: 📝 Los NULLs son aceptables aquí; no todas las descripciones son obligatorias. Si el contenido es vital para un análisis de texto, podrías reemplazar los NULLs con "Sin Descripción".
salary: double(16,2)

Salario de Referencia: El monto salarial asociado a este puesto. ¡Una métrica crucial!
Limpieza: 💰 Revisa que los valores sean numéricos y lógicos. Ceros o valores negativos podrían indicar errores. Si hay NULLs, ¿significa "salario no aplica" o "dato faltante"? Decidir cómo manejar estos NULLs (imputación con promedio/mediana o dejarlos como None) es vital.
status: tinyint(1)

Estado de la Posición: Probablemente un indicador booleano (1 para activa, 0 para inactiva/archivada).
Limpieza: ✅ Asegúrate de que solo contenga 0, 1 o NULL. En Python, esto se traducirá a True/False. Si hay NULLs, la interpretación común es "inactiva" (0).
user_id: int unsigned

Auditoría de Usuario: Posible clave foránea que indica quién creó o modificó el registro.
Limpieza: 🧑‍💻 Útil para trazabilidad. Si es una FK, su integridad es importante para cualquier relación con una tabla de usuarios. NULLs aquí podrían significar un registro antiguo o automático.
created_at: timestamp

Marca de Creación: La fecha y hora exactas en que la posición fue añadida.
Limpieza: ➕ Campo de auditoría. Generalmente limpio y no requiere acción directa, pero valioso para análisis históricos.
updated_at: timestamp

Última Actualización: La última vez que se modificó este registro.
Limpieza: ✏️ Otro campo de auditoría. No requiere limpieza.

🛠️ *Acciones Clave para Limpieza y Transformación*


Para employee_positions, el enfoque principal es la estandarización y la coherencia de los datos, más que el aplanamiento de JSONs.

🏷️ Estandarización de name:

Identifica nombres de puestos similares y unifícalos (ej., "Analista de Datos Jr." y "Analista de Datos Junior" -> "Analista de Datos Junior").
Define una estrategia para NULLs o vacíos en name (reemplazar o marcar).
💲 Gestión de Nulos en salary:

Decide la política para los NULLs: ¿implican ausencia de salario para el rol, o un dato faltante que debería imputarse?
Considera la imputación con la media o mediana si la ausencia de valor es un problema para el análisis.
✅ Conversión y Consistencia de status:

Asegúrate de que se interprete correctamente como booleano (True/False) en Python.
Define cómo manejar los NULLs si aparecen en esta columna.
🔗 Verificación de user_id (Clave Foránea):

Si tienes una tabla de usuarios, es buena práctica verificar que todos los user_id en employee_positions correspondan a un id existente en la tabla de usuarios. Esto asegura la integridad referencial.
📈 Potencial para Tableros BI y Métricas Clave
Esta tabla es una mina de oro para métricas de estructura organizacional y compensación:

Conteo de Puestos Activos: ¿Cuántas posiciones activas tenemos?
Distribución Salarial por Puesto: ¿Cuál es el rango o promedio salarial para cada tipo de posición?
Evolución de Puestos: ¿Cómo han cambiado los nombres de los puestos o la cantidad de roles a lo largo del tiempo? (Combinado con created_at).
Roles Más Comunes: ¿Cuáles son las posiciones con más entradas (si hay duplicados en name o se refiere a instancias de una misma posición)?


✅ *Descripción relacionada a las columnas*:
id: int unsigned (Identificador único de la posición)
name: varchar(255) (Nombre de la posición)
description: varchar(500) (Descripción de la posición)
salary: double(16,2) (Salario asociado a la posición)
status: tinyint(1) (Estado de la posición, probablemente 1 para activa, 0 para inactiva)
user_id: int unsigned (ID del usuario que creó o modificó el registro, parece una clave foránea a una tabla de usuarios)
created_at: timestamp (Marca de tiempo de creación del registro)
updated_at: timestamp (Marca de tiempo de la última actualización del registro)

In [ ]:
# Script para la verificación inicial de datos en 'employee_positions'
try:
    # 1. Obtener el número total de registros
    cursor.execute("SELECT COUNT(*) FROM employee_positions;")
    total_registros = cursor.fetchone()[0]
    print(f"\nTotal de registros en 'employee_positions': {total_registros}")

    # 2. Verificar valores nulos por columna
    print("\nValores nulos por columna en 'employee_positions':")
    for columna_info in columnas_employee_positions: # Usamos 'columnas_employee_positions' que ya obtuvimos
        nombre_columna = columna_info[0]
        cursor.execute(f"SELECT COUNT(*) FROM employee_positions WHERE {nombre_columna} IS NULL;")
        nulos_count = cursor.fetchone()[0]
        if nulos_count > 0:
            print(f"  - {nombre_columna}: {nulos_count} nulos")
        else:
            print(f"  - {nombre_columna}: 0 nulos")

    # 3. Verificar filas duplicadas (basado en todas las columnas, o solo el ID si es la clave primaria)
    # Asumimos que 'id' debería ser único
    cursor.execute("SELECT COUNT(id) - COUNT(DISTINCT id) FROM employee_positions;")
    duplicados_id = cursor.fetchone()[0]
    print(f"\nFilas duplicadas basadas en 'id' en 'employee_positions': {duplicados_id}")

    
    # 4. Obtener valores únicos para columnas específicas
    print("\nValores únicos en columnas clave de 'employee_positions':")

    # Para 'name'
    cursor.execute("SELECT DISTINCT name FROM employee_positions LIMIT 10;") # Limitar para no cargar demasiados si hay muchos
    nombres_unicos = [fila[0] for fila in cursor.fetchall()]
    print(f"  - name (primeros 10 únicos): {nombres_unicos}")
    cursor.execute("SELECT COUNT(DISTINCT name) FROM employee_positions;")
    count_nombres_unicos = cursor.fetchone()[0]
    print(f"  - Total de nombres únicos: {count_nombres_unicos}")

    # Para 'status'
    cursor.execute("SELECT DISTINCT status FROM employee_positions;")
    estados_unicos = [fila[0] for fila in cursor.fetchall()]
    print(f"  - status: {estados_unicos}")


except Exception as e:
    print(f"Ocurrió un error durante la verificación inicial de datos en 'employee_positions': {e}")


Total de registros en 'employee_positions': 15

Valores nulos por columna en 'employee_positions':
  - id: 0 nulos
  - name: 0 nulos
  - description: 0 nulos
  - salary: 0 nulos
  - status: 0 nulos
  - user_id: 0 nulos
  - created_at: 0 nulos
  - updated_at: 0 nulos

Filas duplicadas basadas en 'id' en 'employee_positions': 0

Valores únicos en columnas clave de 'employee_positions':
  - name (primeros 10 únicos): ['Desarrollador', 'General', 'Asistente Administrativo', 'Gerente', 'Tesorero principal', 'Contador General', 'Logístico Integral', 'Diseñador Web', 'Secretaria', 'Auxiliar de soporte']
  - Total de nombres únicos: 14
  - status: [1]


**Bloque 2: Limpieza y Transformación de la 1ra tabla: employee_positions**

In [ ]:
# Script para la limpieza y transformación de 'employee_positions'
try:
    print("\nIniciando limpieza y transformación para 'employee_positions'...")

    # 1. Corrección de errores tipográficos en la columna 'name'
    # Detectamos 'Secretria' que debería ser 'Secretaria'
    print("Corrigiendo error tipográfico en la columna 'name': 'Secretria' a 'Secretaria'...")
    cursor.execute("""
        UPDATE employee_positions
        SET name = 'Secretaria'
        WHERE name = 'Secretria';
    """)
    conexion.commit() # Confirmar los cambios en la base de datos
    print("Corrección de 'name' completada.")

    # 2. Manejo de valores nulos en la columna 'description'
    # Rellenar con 'N/A' según lo acordado.
    print("Rellenando valores nulos en la columna 'description' con 'N/A'...")
    cursor.execute("""
        UPDATE employee_positions
        SET description = 'N/A'
        WHERE description IS NULL;
    """)
    conexion.commit() # Confirmar los cambios
    print("Relleno de 'description' completado.")

    print("\nLimpieza y transformación de 'employee_positions' finalizada.")

except Exception as e:
    print(f"Ocurrió un error durante la limpieza y transformación de 'employee_positions': {e}")
    conexion.rollback() # Revertir cambios en caso de error


Iniciando limpieza y transformación para 'employee_positions'...
Corrigiendo error tipográfico en la columna 'name': 'Secretria' a 'Secretaria'...
Corrección de 'name' completada.
Rellenando valores nulos en la columna 'description' con 'N/A'...
Relleno de 'description' completado.

Limpieza y transformación de 'employee_positions' finalizada.


**TABLA 2: EMPLOYEES o EMPLEADOS**


In [ ]:
# Script para obtener la información de las columnas de 'employees'
try:
    # Obtener la descripción de las columnas de la tabla 'employees'
    cursor.execute("DESCRIBE employees;")
    columnas_employees = cursor.fetchall()

    print("Columnas de la tabla 'employees':")
    for columna in columnas_employees:
        # Cada 'columna' es una tupla con (nombre_columna, tipo_dato, ...)
        print(f"  - {columna[0]} ({columna[1]})")

except Exception as e:
    print(f"Ocurrió un error al intentar obtener las columnas de 'employees': {e}")

Columnas de la tabla 'employees':
  - id (int unsigned)
  - uuid (char(36))
  - identity_document_type (json)
  - document_number (varchar(25))
  - birth_date (date)
  - first_name (varchar(50))
  - other_name (varchar(50))
  - first_surname (varchar(50))
  - second_surname (varchar(50))
  - full_name (varchar(255))
  - phone (varchar(25))
  - email (varchar(75))
  - country (json)
  - region (json)
  - city (json)
  - address (varchar(255))
  - headquarter_id (int unsigned)
  - company_area_id (int unsigned)
  - employee_position_id (int unsigned)
  - emergency_contact (json)
  - payment_type (tinyint unsigned)
  - payment_method (json)
  - bank (json)
  - bank_account_type (json)
  - bank_account_number (varchar(255))
  - status (tinyint(1))
  - settled (tinyint(1))
  - user_id (int unsigned)
  - created_at (timestamp)
  - updated_at (timestamp)


👥 *Análisis Detallado de Variables: Tabla employees*
La tabla employees es el corazón de la información del talento humano. Aquí reside la identidad, contacto y datos clave de cada persona en la organización. Una correcta limpieza y comprensión de sus variables es fundamental para cualquier análisis de Recursos Humanos.

🔑 *Identificadores y Datos Demográficos Clave*
Estas variables nos permiten identificar a cada empleado y entender su perfil básico.
id: INT UNSIGNED
✨ Identificador único principal del empleado. Tu llave maestra para conectar con otros datos.
uuid: CHAR(36)
🔗 Identificador universal único. Piensa en él como un código "global" para integraciones con sistemas externos.
identity_document_type: JSON 🚨
📄 Tipo de documento de identidad (ej: Cédula, Pasaporte). 
document_number: VARCHAR(25)
🔢 Número de identificación.
birth_date: DATE
🎂 Fecha de nacimiento. ¡Indispensable para calcular la edad y realizar análisis demográficos!
first_name: VARCHAR(50)
👤 Primer nombre.
other_name: VARCHAR(50)
✨ Otros nombres. 
first_surname: VARCHAR(50)
apellido.
second_surname: VARCHAR(50)
Segundo apellido. 
full_name: VARCHAR(255) ⚠️
Nombre completo. 
phone: VARCHAR(25)
📞 Número de teléfono de contacto.
email: VARCHAR(75)
📧 Correo electrónico del empleado.

📍 *Información de Ubicación y Estructura Organizacional*
Estos campos nos ubican geográficamente y dentro del organigrama de la empresa.

country: JSON 🚨
🌎 País de residencia. ¡Otro JSON! Requiere aplanamiento para obtener el nombre legible del país.
region: JSON 🚨
🗺️ Región/Departamento/Estado. ¡Un JSON más! Necesita aplanamiento.
city: JSON 🚨
🏙️ Ciudad. ¡Sí, otro JSON a aplanar!
address: VARCHAR(255)
🏠 Dirección física completa.
headquarter_id: INT UNSIGNED
🏢 ID de la sede a la que pertenece el empleado. Es una clave foránea que apunta a la tabla headquarters.
company_area_id: INT UNSIGNED
📊 ID del área/departamento de la compañía. 
employee_position_id: INT 
💼 ID de la posición/cargo del empleado. Clave foránea que conecta con employee_positions.


💳 *Contacto de Emergencia y Detalles Financieros*

Datos cruciales para la gestión interna y operativa del empleado.

emergency_contact: 
🆘 Contacto para emergencias. ¡Es un JSON! Podría contener nombre, relación y número. Es vital aplanarlo o procesarlo para acceder a estos detalles.
payment_type: TINYINT 
💲 Tipo de pago (ej: fijo, por horas). El valor numérico se mapea a una descripción 
payment_method: 
🏦 Método de pago (ej: transferencia bancaria, cheque)
💰 Información del banco del empleado.
bank_account_type: 
💳 Tipo de cuenta bancaria (ej: ahorro, corriente). ¡Un JSON más a aplanar!


🕰️ *Metadatos y Gestión del Registro*
Información de auditoría y estado del registro.

user_id: INT UNSIGNED
🧑‍💻 ID del usuario que creó/modificó el registro.
created_at: TIMESTAMP
➕ Fecha y hora de creación del registro.
updated_at: TIMESTAMP
✏️ Fecha y hora de la última modificación. Crucial para rastrear cambios.
deleted_at: TIMESTAMP 🗑️
Fecha y hora de "borrado lógico". ¡MUY IMPORTANTE! Si esta columna tiene un valor, el empleado no está activo. 
🛠️ Acciones Clave para Limpieza y Transformación en employees
Aplanamiento Masivo de JSONs: La prioridad es transformar todos los campos JSON (identificados con 🚨) en columnas planas y legibles (description o code).
Manejo Inteligente de Nulos: Especial atención a other_name y second_surname. Decide si convertirlos a '' (cadena vacía) o NULL según tu necesidad.
Gestión de full_name: Evalúa si es mejor usar las partes individuales del nombre y concatenarlas, o si confiar en full_name (y si es así, asegurar su consistencia).
Filtrado por Actividad: Implementa el filtro WHERE deleted_at IS NULL para cualquier análisis que requiera solo empleados activos.*

Bloque 4: Verificación inicial de nulos, duplicados y valores únicos en employees


In [18]:
# Importar conexión
from db_connection import get_connection

# Conectarse a la base de datos
conexion = get_connection()
cursor = conexion.cursor()

# Script para la limpieza y transformación inicial de 'employees' (VERSION FINAL)
print("\n--- Bloque 5: Limpieza y Transformación Inicial de 'employees' ---")

try:
    print("Iniciando operaciones de limpieza en 'employees'...")

    # 1. Manejo de valores nulos con 'N/A'
    print("Rellenando valores nulos en 'other_name', 'phone', 'email' con 'N/A'...")
    cursor.execute("""
        UPDATE employees
        SET other_name = 'N/A'
        WHERE other_name IS NULL;
    """)
    conexion.commit() # Confirmar los cambios

    cursor.execute("""
        UPDATE employees
        SET phone = 'N/A'
        WHERE phone IS NULL;
    """)
    conexion.commit() # Confirmar los cambios

    cursor.execute("""
        UPDATE employees
        SET email = 'N/A'
        WHERE email IS NULL;
    """)
    conexion.commit() # Confirmar los cambios
    print("Relleno de nulos completado.")

    # 2. Estandarización de capitalización para nombres
    # Aplicar CONCAT y SUBSTRING solo a los valores que NO sean 'N/A' o cadenas vacías
    print("Estandarizando capitalización de nombres...")

    # first_name
    cursor.execute("""
        UPDATE employees
        SET first_name = CONCAT(UPPER(SUBSTRING(first_name, 1, 1)), LOWER(SUBSTRING(first_name, 2)))
        WHERE first_name IS NOT NULL AND first_name != '';
    """)
    conexion.commit()

    # other_name (solo si no es 'N/A' o vacío)
    cursor.execute("""
        UPDATE employees
        SET other_name = CONCAT(UPPER(SUBSTRING(other_name, 1, 1)), LOWER(SUBSTRING(other_name, 2)))
        WHERE other_name IS NOT NULL AND other_name != 'N/A' AND other_name != '';
    """)
    conexion.commit()

    # first_surname
    cursor.execute("""
        UPDATE employees
        SET first_surname = CONCAT(UPPER(SUBSTRING(first_surname, 1, 1)), LOWER(SUBSTRING(first_surname, 2)))
        WHERE first_surname IS NOT NULL AND first_surname != '';
    """)
    conexion.commit()

    # second_surname (solo si no es 'N/A' o vacío)
    cursor.execute("""
        UPDATE employees
        SET second_surname = CONCAT(UPPER(SUBSTRING(second_surname, 1, 1)), LOWER(SUBSTRING(second_surname, 2)))
        WHERE second_surname IS NOT NULL AND second_surname != 'N/A' AND second_surname != '';
    """)
    conexion.commit()

    # full_name (solo si no es 'N/A' o vacío)
    cursor.execute("""
        UPDATE employees
        SET full_name = CONCAT(UPPER(SUBSTRING(full_name, 1, 1)), LOWER(SUBSTRING(full_name, 2)))
        WHERE full_name IS NOT NULL AND full_name != 'N/A' AND full_name != '';
    """)
    conexion.commit()

    print("Estandarización de capitalización completada.")

    print("\nLimpieza y transformación inicial de 'employees' finalizada exitosamente.")

except Exception as e:
    print(f"Ocurrió un error inesperado durante la limpieza y transformación inicial de 'employees': {e}")
    # En caso de error, intentar limpiar el cursor y revertir.
    try:
        # Asegurar que cualquier resultado pendiente sea consumido antes de rollback
        if cursor.description is not None:
            cursor.fetchall()
        conexion.rollback()
        print("Transacción revertida debido a un error.")
    except Exception as rollback_e:
        print(f"Error adicional al intentar revertir la transacción o limpiar el cursor: {rollback_e}")

✅ Conexión exitosa a la base de datos

--- Bloque 5: Limpieza y Transformación Inicial de 'employees' ---
Iniciando operaciones de limpieza en 'employees'...
Rellenando valores nulos en 'other_name', 'phone', 'email' con 'N/A'...
Relleno de nulos completado.
Estandarizando capitalización de nombres...
Estandarización de capitalización completada.

Limpieza y transformación inicial de 'employees' finalizada exitosamente.


💰  *Análisis realizado para la exploración en la segunda tabla y acciones a tomar* 

La tabla contract_salary_history es vital para comprender la evolución salarial de los empleados a lo largo del tiempo. Contiene un registro de los cambios en los salarios asociados a los contratos, permitiendo analizar tendencias y el impacto de las revisiones salariales.

🔑 *Identificadores y Vínculos Clave*

Estas variables nos permiten identificar cada registro histórico y vincularlo con el contrato y el empleado al que corresponde.

id: INT UNSIGNED
✨ Identificador único principal del registro histórico de salario. La clave de cada entrada en esta tabla.
contract_id: INT UNSIGNED
🔗 Identificador del contrato al que pertenece este registro salarial. Es una clave foránea que te conectará directamente con la tabla employee_contracts. ¡Esencial para el análisis contextual!
employee_id: INT UNSIGNED
👤 Identificador del empleado asociado a este historial salarial. Otra clave foránea que te vincula directamente con la tabla employees. Permite un seguimiento del historial salarial por cada persona.

💵 *Detalles Salariales y Vigencia*

Estos campos registran el salario en un punto específico del tiempo y su duración.

salary: DECIMAL(18,2)
💲 El monto del salario registrado. Es crucial para el análisis de la evolución salarial. Asegúrate de que los valores numéricos estén limpios y sean coherentes.
start_date: DATE
🗓️ Fecha de inicio de la vigencia de este salario. Determina cuándo entró en efecto el cambio salarial.
end_date: DATE
🗓️ Fecha de fin de la vigencia de este salario. ¡Atención a los Nulos! Un valor NULL en esta columna generalmente indica que este es el salario actual y vigente.


🕰️ *Metadatos y Auditoría*
Información para la trazabilidad y gestión del registro.

user_id: INT UNSIGNED
🧑‍💻 ID del usuario que creó o modificó este registro salarial. Útil para auditoría.
created_at: TIMESTAMP
➕ Fecha y hora de creación del registro. Indica cuándo se añadió este historial a la base de datos.
updated_at: TIMESTAMP
✏️ Fecha y hora de la última modificación del registro. Permite rastrear la última vez que se actualizó esta entrada.

🛠️ *Acciones Clave para Limpieza y Transformación*

Para la tabla contract_salary_history, las acciones de limpieza son más sencillas en comparación con tablas con JSONs. La prioridad será la coherencia de fechas y el manejo de nulos.

Manejo de Nulos en end_date: Para análisis de salarios activos, los registros donde end_date es NULL representan el salario actual. Dependiendo del análisis, podrías querer reemplazar NULL con una fecha futura lejana o simplemente filtrar por ellos.
Validación de Rangos de Fechas: Asegúrate de que start_date siempre sea anterior a end_date (cuando esta última existe).
Consistencia Salarial: Aunque esta tabla registra el historial, podrías querer hacer un chequeo de coherencia básico en la columna salary (por ejemplo, si hay valores negativos o ceros que no deberían existir).
No se esperan JSONs: Según la descripción proporcionada, esta tabla no parece contener columnas de tipo JSON, lo que simplifica el proceso de aplanamiento.

In [28]:
# Script para limpiar y aplanar columnas JSON en 'employees' (VERSIÓN CON CONEXIÓN ABIERTA)

import pandas as pd
import json
import numpy as np
from db_connection import get_engine, get_connection

print("\n--- Bloque 6 (VERSIÓN CON CONEXIÓN ABIERTA): Limpieza de JSONs en 'employees' con Pandas ---")

conexion = None
cursor = None
try:
    print("Reestableciendo conexión a la base de datos para el bloque de 'employees'...")
    conexion = get_connection()
    cursor = conexion.cursor()
    engine = get_engine()
    print("✅ Conexión y cursor reestablecidos exitosamente para 'employees'.")
    print("✅ Engine de base de datos obtenido.")

    # --- PASO CRÍTICO: Deshabilitar temporalmente las verificaciones de claves foráneas ---
    print("\nDeshabilitando temporalmente las verificaciones de claves foráneas...")
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")
    print("✅ Verificaciones de claves foráneas deshabilitadas.")

    # --- PASO 1: Identificar y corregir IDs huérfanos en `company_area_id` ---
    print("\nBuscando IDs huérfanos en 'company_area_id' de la tabla 'employees'...")
    # Consulta para encontrar IDs huérfanos
    cursor.execute("""
        SELECT DISTINCT e.company_area_id
        FROM employees e
        LEFT JOIN company_areas ca ON e.company_area_id = ca.id
        WHERE ca.id IS NULL AND e.company_area_id IS NOT NULL;
    """)
    orphaned_company_area_ids = [row[0] for row in cursor.fetchall()]

    if orphaned_company_area_ids:
        print(f"⚠️ Se encontraron IDs huérfanos en 'company_area_id': {orphaned_company_area_ids}")

        # --- AHORA, NECESITAS UN ID VÁLIDO DE 'company_areas' PARA ASIGNAR ---
        # Si tienes un ID específico en mente (ej. un área 'Otros' o 'Sin Asignar'), úsalo aquí.
        # Si no, puedes buscar el primer ID válido, pero lo ideal es tener uno específico para valores no válidos.
        # EJEMPLO: Asumiendo que 1 es un ID válido en company_areas. CAMBIA ESTO SEGÚN TUS DATOS.
        valid_default_company_area_id = 1 # <--- CAMBIA ESTE VALOR POR UN ID VÁLIDO EN company_areas
        print(f"Asignando '{valid_default_company_area_id}' a los registros huérfanos.")

        # Actualizar los registros con IDs huérfanos
        # Usamos %s para cada ID huérfano para seguridad contra inyección SQL
        placeholders = ','.join(['%s'] * len(orphaned_company_area_ids))
        update_orphan_sql = f"""
            UPDATE employees
            SET company_area_id = %s
            WHERE company_area_id IN ({placeholders});
        """
        try:
            # Los valores se pasan como una tupla: (el_nuevo_id, id_huerfano1, id_huerfano2, ...)
            cursor.execute(update_orphan_sql, (valid_default_company_area_id, *orphaned_company_area_ids))
            conexion.commit() # Confirmar esta operación antes del ALTER TABLE
            print(f"✅ Se actualizaron {cursor.rowcount} registros de 'employees' con IDs huérfanos.")
        except Exception as e:
            conexion.rollback()
            print(f"❌ Error al actualizar IDs huérfanos: {e}")
            raise # Re-lanza el error para detener el script si falla esta parte
    else:
        print("✅ No se encontraron IDs huérfanos en 'company_area_id'.")

    # --- PASO 2: Modificar la estructura de la tabla 'employees' ---
    # Convertir las columnas JSON a VARCHAR(255) NULL
    print("\nModificando las columnas JSON a VARCHAR(255) NULL...")
    alter_table_sql = """
        ALTER TABLE employees
        MODIFY COLUMN identity_document_type VARCHAR(255) NULL,
        MODIFY COLUMN country VARCHAR(255) NULL,
        MODIFY COLUMN region VARCHAR(255) NULL,
        MODIFY COLUMN city VARCHAR(255) NULL,
        MODIFY COLUMN emergency_contact VARCHAR(255) NULL,
        MODIFY COLUMN payment_method VARCHAR(255) NULL,
        MODIFY COLUMN bank VARCHAR(255) NULL,
        MODIFY COLUMN bank_account_type VARCHAR(255) NULL;
    """
    try:
        cursor.execute(alter_table_sql)
        conexion.commit() # Confirmar el cambio de estructura
        print("✅ Columnas JSON modificadas a VARCHAR(255) NULL exitosamente.")
    except Exception as e:
        conexion.rollback()
        print(f"❌ Error al modificar las columnas JSON: {e}")
        raise # Re-lanza el error

    # --- PASO CRÍTICO: Re-habilitar las verificaciones de claves foráneas ---
    print("\nRe-habilitando las verificaciones de claves foráneas...")
    cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")
    print("✅ Verificaciones de claves foráneas re-habilitadas.")

    # 1. Cargar la tabla 'employees' completa en un DataFrame de Pandas
    print("\nCargando la tabla 'employees' en un DataFrame de Pandas...")
    df_employees = pd.read_sql("SELECT * FROM employees;", engine)
    print(f"DataFrame de 'employees' cargado. Registros: {len(df_employees)}")

    # Lista de columnas JSON a procesar (ahora estas columnas son VARCHAR en la DB)
    json_columns = [
        'identity_document_type', 'country', 'region', 'city',
        'emergency_contact', 'payment_method', 'bank', 'bank_account_type'
    ]

    # 2. Iterar sobre cada columna (ahora VARCHAR, pero con contenido JSON previo) y aplanarla
    print("\nIniciando aplanamiento de columnas (previamente JSON) en Pandas...")

    def extract_json_value(json_str):
        # Manejar casos donde la columna ya fue aplanada o es None/NaN
        if pd.isna(json_str) or not isinstance(json_str, str) or not json_str.strip():
            return None
        # Si la cadena no parece JSON (ej. ya es un valor aplanado), devolverla tal cual
        if not json_str.strip().startswith('{') and not json_str.strip().startswith('['):
            return json_str # Ya está aplanado, o es un string simple no JSON
        try:
            data = json.loads(json_str)
            if isinstance(data, dict):
                if 'description' in data and data['description'] is not None:
                    return str(data['description'])
                elif 'name' in data and data['name'] is not None:
                    return str(data['name'])
                elif 'code' in data and data['code'] is not None:
                    return str(data['code'])
            return None
        except json.JSONDecodeError:
            # Si no es un JSON válido, pero es una cadena, devolverla.
            # Esto maneja el caso donde la columna ya fue aplanada manualmente o contiene basura.
            return json_str if json_str.strip() else None
        except Exception as e:
            return None

    for col in json_columns:
        print(f"Procesando columna: '{col}'...")
        if col in df_employees.columns:
            df_employees[col] = df_employees[col].astype(str)
            df_employees[col] = df_employees[col].apply(extract_json_value)
            print(f"   Columna '{col}' aplanada a valor extraído o NULL.")
        else:
            print(f"   Advertencia: Columna '{col}' no encontrada en el DataFrame.")
    print("✅ Aplanamiento de columnas completado.")

    # 3. Manejo de valores nulos y tipos de datos para la actualización
    print("\nPreparando DataFrame para la actualización (manejo de nulos y tipos)...")

    # Volvemos a obtener la descripción de la tabla para asegurarnos de que los tipos son los nuevos (VARCHAR)
    cursor.execute("DESCRIBE `employees`;")
    db_columns_info = cursor.fetchall()
    db_column_names = [col_info[0] for col_info in db_columns_info]
    db_column_types = {col_info[0]: col_info[1] for col_info in db_columns_info}

    valid_columns_for_update = [
        col for col in df_employees.columns # <--- LINEA CORREGIDA AQUI
        if isinstance(col, str) and col != 'id' and col in db_column_names
    ]

    for col in df_employees.columns:
        if col in valid_columns_for_update or col == 'id':
            if df_employees[col].dtype == 'object':
                df_employees[col] = df_employees[col].replace({np.nan: None, 'nan': None, '': None})
            elif pd.api.types.is_numeric_dtype(df_employees[col]) and df_employees[col].isnull().any():
                if pd.api.types.is_integer_dtype(df_employees[col]):
                     df_employees[col] = df_employees[col].astype(pd.Int64Dtype())
                df_employees[col] = df_employees[col].replace({np.nan: None})

            if pd.api.types.is_bool_dtype(df_employees[col]):
                df_employees[col] = df_employees[col].map({True: 1, False: 0, None: None, np.nan: None})

            if pd.api.types.is_string_dtype(df_employees[col]) and col in db_column_types:
                mysql_type = db_column_types[col].lower()
                if 'varchar' in mysql_type:
                    try:
                        max_len = int(mysql_type.split('(')[1].split(')')[0])
                        long_strings = df_employees[df_employees[col].astype(str).str.len() > max_len][col]
                        if not long_strings.empty:
                            print(f"   ⚠️ ADVERTENCIA: Columna '{col}' tiene strings más largos que {max_len} caracteres.")
                            print(f"     Primeros 5 valores problemáticos: {long_strings.head().tolist()}")
                    except (IndexError, ValueError):
                        pass

    print("✅ DataFrame preparado para la actualización.")

    # 4. Actualizar los registros con los datos limpios del DataFrame
    print("\n--- Actualizando registros en 'employees' con datos limpios ---")

    set_clauses = [f"`{col}` = %s" for col in valid_columns_for_update]
    update_sql = f"UPDATE `employees` SET {', '.join(set_clauses)} WHERE `id` = %s;"

    print(f"DEBUG: SQL de UPDATE generado: {update_sql[:150]}...")
    print(f"DEBUG: Columnas que se actualizarán: {valid_columns_for_update}")

    updated_count = 0
    errors_encountered = 0
    for index, row in df_employees.iterrows():
        update_values = []
        for col in valid_columns_for_update:
            val = row[col]
            if pd.isna(val) or (isinstance(val, (float, np.float64)) and np.isnan(val)):
                update_values.append(None)
            elif isinstance(val, (np.int64, np.int32)):
                update_values.append(int(val))
            elif isinstance(val, (np.bool_)):
                update_values.append(bool(val))
            else:
                update_values.append(val)

        update_values.append(row['id'])

        try:
            cursor.execute(update_sql, tuple(update_values))
            updated_count += 1
            if updated_count % 5 == 0:
                print(f"   Progreso: {updated_count}/{len(df_employees)} registros procesados...")
        except Exception as e:
            errors_encountered += 1
            print(f"\n❌❌ ERROR CRÍTICO al actualizar el registro con ID {row['id']}: {e}")
            print(f"   SQL: {update_sql}")
            print(f"   Valores problemáticos (ID={row['id']}): {tuple(update_values)}")
            raise

    if errors_encountered > 0:
        conexion.rollback()
        print(f"❌ Actualización finalizada con {errors_encountered} errores. Transacción revertida.")
    else:
        conexion.commit()
        print(f"✅ Actualización de {updated_count} registros en la tabla 'employees' completada exitosamente.")

    print("\n✅ Limpieza y aplanamiento de JSONs en 'employees' finalizada exitosamente.")

except Exception as e:
    print(f"\n❌❌ Ocurrió un error general NO CAPTURADO en el bucle principal: {e}")
    print("Por favor, revisa la conexión a la base de datos o los datos en sí.")
    try:
        if conexion and conexion.is_connected():
            conexion.rollback()
            print("Transacción revertida debido a un error.")
    except Exception as rollback_e:
        print(f"❌ Error adicional al intentar revertir la transacción: {rollback_e}")
finally:
    # Asegúrate de re-habilitar las FKs incluso si hay un error en el try-except
    if cursor is not None:
        try:
            cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")
            print("✅ Verificaciones de claves foráneas re-habilitadas en el bloque 'finally'.")
        except Exception as e:
            print(f"❌ Advertencia: No se pudieron re-habilitar las FKs en finally: {e}")

    # No cierres la conexión aquí si quieres mantenerla abierta como indicaste antes
    print("La conexión a la base de datos se mantiene abierta según la solicitud.")


--- Bloque 6 (VERSIÓN CON CONEXIÓN ABIERTA): Limpieza de JSONs en 'employees' con Pandas ---
Reestableciendo conexión a la base de datos para el bloque de 'employees'...
✅ Conexión exitosa a la base de datos
✅ Conexión y cursor reestablecidos exitosamente para 'employees'.
✅ Engine de base de datos obtenido.

Deshabilitando temporalmente las verificaciones de claves foráneas...
✅ Verificaciones de claves foráneas deshabilitadas.

Buscando IDs huérfanos en 'company_area_id' de la tabla 'employees'...

❌❌ Ocurrió un error general NO CAPTURADO en el bucle principal: 1146 (42S02): Table 'novateam.company_areas' doesn't exist
Por favor, revisa la conexión a la base de datos o los datos en sí.
Transacción revertida debido a un error.
✅ Verificaciones de claves foráneas re-habilitadas en el bloque 'finally'.
La conexión a la base de datos se mantiene abierta según la solicitud.


In [29]:
# Script para limpiar y aplanar columnas JSON en 'employees' usando Pandas 

import pandas as pd
from db_connection import get_engine # Asegúrate de tener get_engine configurado

print("\n--- Bloque 6 (Estrategia Definitiva): Limpieza de JSONs en 'employees' con Pandas ---")

try:
    # 1. Obtener el engine de SQLAlchemy
    engine = get_engine()
    print("Engine de base de datos obtenido.")

    # 2. Cargar la tabla 'employees' completa en un DataFrame de Pandas
    print("Cargando la tabla 'employees' en un DataFrame de Pandas...")
    df_employees = pd.read_sql("SELECT * FROM employees;", engine)
    print(f"DataFrame de 'employees' cargado. Registros: {len(df_employees)}")

    # Lista de columnas JSON a procesar
    json_columns = [
        'identity_document_type',
        'country',
        'region',
        'city',
        'emergency_contact',
        'payment_method',
        'bank',
        'bank_account_type'
    ]

    # 3. Iterar sobre cada columna JSON y aplanarla en Pandas
    print("\nIniciando aplanamiento de columnas JSON en Pandas...")
    for col in json_columns:
        print(f"Procesando columna JSON: '{col}'...")
        
        # Función para extraer la descripción de un JSON en Python, manejando errores
        def extract_json_description(json_str):
            if pd.isna(json_str) or not isinstance(json_str, str):
                return 'N/A' # Manejar NaN o tipos no string
            try:
                # Intentar cargar como JSON. Si falla, es inválido.
                # json.loads() es más estricto y útil para capturar el error.
                import json
                data = json.loads(json_str)
                # Si es un objeto JSON y tiene 'description', la extraemos
                if isinstance(data, dict) and 'description' in data:
                    return str(data['description'])
                # Si es un objeto pero no tiene 'description', devolvemos 'N/A'
                return 'N/A'
            except json.JSONDecodeError:
                # Si no es un JSON válido o la carga falla, devolver 'N/A'
                print(f"    Advertencia: Valor inválido detectado en '{col}': '{json_str}'. Se reemplazará con 'N/A'.")
                return 'N/A'
            except Exception as e:
                # Otros errores inesperados durante el procesamiento del JSON
                print(f"    Advertencia: Error inesperado al procesar '{json_str}' en '{col}': {e}. Se reemplazará con 'N/A'.")
                return 'N/A'
        
        # Aplicar la función a la columna
        df_employees[col] = df_employees[col].apply(extract_json_description)
        print(f"  Columna '{col}' aplanada a 'description' o 'N/A'.")

    print("\nAplanamiento de JSONs en DataFrame completado.")

    # 4. Volver a guardar el DataFrame limpio en la base de datos
    # ¡ADVERTENCIA!: 'if_exists='replace'' BORRARÁ Y RECREARÁ LA TABLA.
    # Esta es la forma más directa de asegurar que los cambios se reflejen.
    print("Volcando el DataFrame limpio de vuelta a la tabla 'employees' en la base de datos...")
    df_employees.to_sql(name='employees', con=engine, if_exists='replace', index=False)
    print("DataFrame volcado a la base de datos exitosamente.")

    print("\nLimpieza y aplanamiento de JSONs en 'employees' finalizada exitosamente.")

except Exception as e:
    print(f"Ocurrió un error general durante el limpieza de JSONs en 'employees' con Pandas: {e}")
    print("Por favor, revisa la conexión a la base de datos o los datos en sí.")


--- Bloque 6 (Estrategia Definitiva): Limpieza de JSONs en 'employees' con Pandas ---
Engine de base de datos obtenido.
Cargando la tabla 'employees' en un DataFrame de Pandas...
DataFrame de 'employees' cargado. Registros: 38

Iniciando aplanamiento de columnas JSON en Pandas...
Procesando columna JSON: 'identity_document_type'...
    Advertencia: Valor inválido detectado en 'identity_document_type': 'Cédula de ciudadanía'. Se reemplazará con 'N/A'.
    Advertencia: Valor inválido detectado en 'identity_document_type': 'Cédula de ciudadanía'. Se reemplazará con 'N/A'.
    Advertencia: Valor inválido detectado en 'identity_document_type': 'Cédula de ciudadanía'. Se reemplazará con 'N/A'.
    Advertencia: Valor inválido detectado en 'identity_document_type': 'Cédula de ciudadanía'. Se reemplazará con 'N/A'.
    Advertencia: Valor inválido detectado en 'identity_document_type': 'Cédula de ciudadanía'. Se reemplazará con 'N/A'.
    Advertencia: Valor inválido detectado en 'identity_docum

*TABLA 3: employee_contracts o Contrato de empleados*


In [30]:
# Script para obtener la información de las columnas de 'employee_contracts' (RE-REVISADO 2)

# Asegúrate de que get_connection esté importado desde tu db_connection.py
from db_connection import get_connection

print("\n--- Bloque 7: Exploración de la Estructura de 'employee_contracts' ---")

# Reestablecer la conexión y el cursor
print("Reestableciendo conexión a la base de datos...")
try:
    # Cerrar si hay alguna conexión o cursor previo que no se cerró correctamente
    # SOLO la conexión tiene is_connected()
    if 'conexion' in globals() and conexion is not None and conexion.is_connected():
        # Primero cierra el cursor si existe y está asociado a la conexión
        if 'cursor' in globals() and cursor is not None:
            try:
                cursor.close()
                del cursor # Eliminar la variable del cursor
            except Exception as e:
                print(f"Advertencia al cerrar cursor previo: {e}")
        conexion.close()
        del conexion # Eliminar la variable de la conexión
    
    conexion = get_connection()
    cursor = conexion.cursor()
    print("Conexión y cursor reestablecidos.")
except Exception as e:
    print(f"ERROR: No se pudo reestablecer la conexión a la base de datos: {e}")
    # Si no podemos conectar, no tiene sentido continuar
    raise

try:
    cursor.execute("DESCRIBE employee_contracts;")
    columnas_employee_contracts = cursor.fetchall()

    print("Columnas de la tabla 'employee_contracts':")
    for columna in columnas_employee_contracts:
        print(f"  - {columna[0]} ({columna[1]})")

except Exception as e:
    print(f"Ocurrió un error al intentar obtener las columnas de 'employee_contracts': {e}")
finally:
    # Cerrar la conexión y el cursor al finalizar el bloque para limpiar recursos
    # Asegurarse de que las variables existan antes de intentar cerrarlas
    if 'cursor' in locals() and cursor is not None:
        try:
            cursor.close()
        except Exception as e:
            print(f"Advertencia al cerrar cursor en finally: {e}")
    if 'conexion' in locals() and conexion is not None and conexion.is_connected():
        try:
            conexion.close()
        except Exception as e:
            print(f"Advertencia al cerrar conexión en finally: {e}")
    print("Conexión cerrada al finalizar el bloque.")


--- Bloque 7: Exploración de la Estructura de 'employee_contracts' ---
Reestableciendo conexión a la base de datos...
✅ Conexión exitosa a la base de datos
Conexión y cursor reestablecidos.
Columnas de la tabla 'employee_contracts':
  - id (int unsigned)
  - employee_id (int unsigned)
  - employee_type (varchar(255))
  - employee_subtype (varchar(255))
  - high_penson_risk (tinyint(1))
  - integral_salary (tinyint(1))
  - type (varchar(255))
  - start_date (date)
  - end_date (date)
  - salary (double(16,2))
  - transport_subsidy (tinyint(1))
  - risk_type (varchar(255))
  - holidays_enjoyed (smallint unsigned)
  - health_provider_id (bigint unsigned)
  - health_provider (varchar(255))
  - pension_provider (varchar(255))
  - pension_provider_id (bigint unsigned)
  - severance_provider (varchar(255))
  - severance_provider_id (bigint unsigned)
  - family_compensation_fund_provider (json)
  - family_compensation_fund_provider_id (bigint unsigned)
  - settlement_cause (varchar(255))
  - 

📜 *Análisis Detallado de la Tabla employee_contracts*


🎯 Propósito y Relevancia Estratégica en RRHH

Esta tabla es el repositorio de todos los acuerdos laborales activos e históricos. Cada fila representa un contrato específico, detallando los términos de empleo, el salario, la duración y los proveedores asociados. Es fundamental para:

Gestión Contractual: Monitorear tipos de contratos y su vigencia.
Análisis de Compensación: Vincular salarios a tipos de contrato y periodos.
Beneficios y Seguridad Social: Rastrear proveedores de salud, pensión y cesantías.
Cálculo de Antigüedad y Rotación: Las fechas de inicio y fin son clave aquí.

🔍 *Columnas Clave y Consideraciones Cruciales*

id: int unsigned
Identificador Único: La ✨ clave primaria ✨ de la tabla. Un id único para cada contrato asegura su singularidad.
Limpieza: No requiere limpieza. Es tu núcleo de referencia para este contrato.
employee_id: int unsigned

Vínculo con el Empleado: Es una 🔗 clave foránea que une este contrato con su respectivo empleado en la tabla employees.
Limpieza: ¡Cuidado! Es un punto crítico de integridad. Debemos verificar que todos los employee_id existan en employees. Si hay IDs huérfanos, necesitaríamos la estrategia de deshabilitar/re-habilitar FOREIGN_KEY_CHECKS para cualquier corrección.
employee_type: json

Tipo de Empleado: Categoriza la naturaleza de la vinculación (ej., "fijo", "temporal").
Limpieza: Es una columna JSON que debes aplanar. Extraeremos el valor relevante (como description, name o code) a un VARCHAR(255) para facilitar su uso en análisis.
employee_subtype: json

Subtipo de Empleado: Provee una categoría más granular si aplica.
Limpieza: Otra columna JSON para aplanar, siguiendo la misma lógica que employee_type.
high_penson_risk: tinyint(1)

Indicador de Riesgo: Booleano que señala si hay un alto riesgo de pensión.
Limpieza: ✅ MySQL lo almacena como 0 o 1. Python lo interpretará como False/True. Si hay NULLs, la decisión es si convertirlos a 0 (Falso) o mantenerlos como None.
integral_salary: tinyint(1)

Salario Integral: Booleano que indica si el salario del contrato es de tipo integral.
Limpieza: Similar a high_penson_risk, maneja los NULLs si existen.
type: json

Tipo de Contrato: Define la modalidad del contrato (ej., "a término indefinido", "obra labor").
Limpieza: Una columna JSON vital para aplanar. Es fundamental para cualquier clasificación contractual.
start_date: date

Inicio del Contrato: La fecha en que el contrato comenzó su vigencia.
Limpieza: 🗓️ Debería ser un campo obligatorio. Asegúrate de que los formatos de fecha sean válidos. Los NULLs serían anómalos.
end_date: date

Fin del Contrato: La fecha de finalización del contrato.
Limpieza: 🛑 ¡Atención especial! Es completamente normal que sea NULL para contratos indefinidos o que aún están activos. Al calcular la duración del contrato, los NULLs se interpretarán como "hasta la fecha actual" o "vigente".
salary: double(16,2)

Monto Salarial: El salario acordado en este contrato.
Limpieza: 💲 Revisa valores atípicos (salarios de 0 o inusuales). Si hay NULLs, y el salario es obligatorio, pueden necesitar imputación o reemplazo.
transport_subsidy: tinyint(1)

Subsidio de Transporte: Booleano que indica si se otorga subsidio de transporte.
Limpieza: Maneja los NULLs si existen.
risk_type: json

Tipo de Riesgo Laboral: Clasifica el riesgo asociado a la labor.
Limpieza: Columna JSON para aplanar. Es importante para análisis de seguridad y salud.
holidays_enjoyed: smallint unsigned

Días de Vacaciones: Registra los días de vacaciones que el empleado ha disfrutado bajo este contrato.
Limpieza: 🏖️ Los NULLs podrían significar 0 días disfrutados o dato no disponible. La decisión de convertir NULL a 0 depende de la lógica de negocio.
health_provider_id: bigint unsigned

ID Proveedor Salud: Posible 🔗 clave foránea al proveedor de salud.
Limpieza: Si es FK, verifica su integridad con la tabla de proveedores de salud.
health_provider: json

Info. Proveedor Salud: Detalles del proveedor de salud.
Limpieza: Columna JSON para aplanar.
pension_provider: json

Info. Proveedor Pensión: Detalles del proveedor de pensión.
Limpieza: Columna JSON para aplanar.
pension_provider_id: bigint unsigned

ID Proveedor Pensión: Posible 🔗 clave foránea al proveedor de pensión.
Limpieza: Si es FK, verifica su integridad con la tabla de proveedores de pensión.
severance_provider: json

Info. Proveedor Cesantías: Detalles del proveedor de cesantías.
Limpieza: Columna JSON para aplanar.
severance_provider_id: bigint unsigned

ID Proveedor Cesantías: Posible 🔗 clave foránea al proveedor de cesantías.
Limpieza: Si es FK, verifica su integridad con la tabla de proveedores de cesantías.
settlement_approved_at: timestamp

Aprobación Liquidación: Fecha de aprobación de la liquidación del contrato.
Limpieza: ⏳ ¡Punto crítico! Será NULL para contratos activos. Solo tendrá valor para contratos que ya han finalizado y han sido liquidados. Un NULL aquí es informativo.
created_at: timestamp

Marca de Creación: Fecha y hora en que el registro fue creado.
Limpieza: ➕ Campo de auditoría. Generalmente no requiere limpieza.
updated_at: timestamp

Última Actualización: Fecha y hora de la última modificación del registro.
Limpieza: ✏️ Campo de auditoría. No requiere limpieza.

🛠️ *Acciones Clave para Limpieza y Transformación*


La limpieza de employee_contracts girará principalmente en torno al aplanamiento de JSONs y el manejo inteligente de las fechas y claves foráneas.

Aplanar Columnas JSON:

employee_type, employee_subtype, type, risk_type, health_provider, pension_provider, severance_provider.
Aplicaremos la misma función extract_json_value que ya usamos. Esto requerirá un ALTER TABLE para cambiar su tipo a VARCHAR(255) NULL, usando la estrategia de deshabilitar/re-habilitar FOREIGN_KEY_CHECKS.
Manejo Inteligente de Fechas (end_date, settlement_approved_at):

end_date: Reconocer que NULL es un valor significativo (contrato activo). Para cálculos de duración, sustituir NULL con la fecha actual.
settlement_approved_at: NULL también es significativo (contrato no liquidado).
Conversión de Booleanos:

Asegurar que high_penson_risk, integral_salary, transport_subsidy se manejen como booleanos en Pandas, gestionando posibles NULLs.
Verificación de Claves Foráneas (FKs):

employee_id: Prioridad alta. Identificar y corregir cualquier employee_id huérfano antes de cualquier operación que pueda afectar la integridad.
IDs de Proveedores (health_provider_id, pension_provider_id, severance_provider_id): Si estas son FKs, la misma lógica de verificación y corrección se aplica si hay inconsistencias.


📈 *Potencial para Tableros BI y Métricas Clave*
Esta tabla es una fuente inagotable de insights para RRHH:

📊 Distribución por Tipo de Contrato: ¿Cuántos contratos de cada tipo tenemos (indefinido, obra labor, etc.)?
⏳ Duración Promedio de Contratos: ¿Cuánto tiempo duran nuestros contratos a término fijo?
💸 Salario Promedio por Tipo de Contrato: Comparar la compensación entre diferentes modalidades contractuales.
📈 Rotación de Contratos: Analizar cuántos contratos finalizan en un periodo y por qué (si se combina con otras fuentes).
⚕️ Análisis de Proveedores: ¿Qué proveedores de salud, pensión o cesantías son los más utilizados?


In [ ]:
# Script para la verificación inicial de datos en 'employee_contracts'

# Asegúrate de que get_connection esté importado desde tu db_connection.py
from db_connection import get_connection

print("\n--- Bloque 8: Verificación Inicial de 'employee_contracts' ---")

# Reestablecer la conexión y el cursor para este bloque
print("Reestableciendo conexión a la base de datos...")
try:
    if 'conexion' in globals() and conexion is not None and conexion.is_connected():
        if 'cursor' in globals() and cursor is not None:
            try:
                cursor.close()
                del cursor
            except Exception as e:
                print(f"Advertencia al cerrar cursor previo: {e}")
        conexion.close()
        del conexion
    
    conexion = get_connection()
    cursor = conexion.cursor()
    print("Conexión y cursor reestablecidos.")
except Exception as e:
    print(f"ERROR: No se pudo reestablecer la conexión a la base de datos: {e}")
    raise

try:
    # 1. Obtener el número total de registros
    cursor.execute("SELECT COUNT(*) FROM employee_contracts;")
    total_registros_contracts = cursor.fetchone()[0]
    print(f"\nTotal de registros en 'employee_contracts': {total_registros_contracts}")

    # 2. Verificar valores nulos por columna
    print("\nValores nulos por columna en 'employee_contracts':")
    # Para esto, usaremos la lista de columnas que acabas de compartir
    # (asumiendo que 'columnas_employee_contracts' está disponible o se recrea)
    cursor.execute("DESCRIBE employee_contracts;")
    columnas_employee_contracts = cursor.fetchall()
    
    for columna_info in columnas_employee_contracts:
        nombre_columna = columna_info[0]
        cursor.execute(f"SELECT COUNT(*) FROM employee_contracts WHERE {nombre_columna} IS NULL;")
        nulos_count = cursor.fetchone()[0]
        if nulos_count > 0:
            print(f"  - {nombre_columna}: {nulos_count} nulos")
        else:
            print(f"  - {nombre_columna}: 0 nulos")

    # 3. Verificar filas duplicadas
    cursor.execute("SELECT COUNT(id) - COUNT(DISTINCT id) FROM employee_contracts;")
    duplicados_id_contracts = cursor.fetchone()[0]
    print(f"\nFilas duplicadas basadas en 'id' en 'employee_contracts': {duplicados_id_contracts}")

    # Es posible que un empleado tenga múltiples contratos a lo largo del tiempo,
    # pero no debería tener dos contratos con la misma fecha de inicio.
    cursor.execute("""
        SELECT COUNT(*)
        FROM (
            SELECT employee_id, start_date, COUNT(*)
            FROM employee_contracts
            GROUP BY employee_id, start_date
            HAVING COUNT(*) > 1
        ) AS duplicated_contracts;
    """)
    duplicados_employee_date = cursor.fetchone()[0]
    print(f"Contratos duplicados basados en 'employee_id' y 'start_date': {duplicados_employee_date}")

    # 4. Obtener valores únicos para columnas booleanas (tinyint) y una muestra de JSON
    print("\nValores únicos en columnas clave de 'employee_contracts':")

    # Para 'high_penson_risk'
    cursor.execute("SELECT DISTINCT high_penson_risk FROM employee_contracts;")
    high_penson_risk_unicos = [fila[0] for fila in cursor.fetchall()]
    print(f"  - high_penson_risk: {high_penson_risk_unicos}")

    # Para 'integral_salary'
    cursor.execute("SELECT DISTINCT integral_salary FROM employee_contracts;")
    integral_salary_unicos = [fila[0] for fila in cursor.fetchall()]
    print(f"  - integral_salary: {integral_salary_unicos}")

    # Para 'transport_subsidy'
    cursor.execute("SELECT DISTINCT transport_subsidy FROM employee_contracts;")
    transport_subsidy_unicos = [fila[0] for fila in cursor.fetchall()]
    print(f"  - transport_subsidy: {transport_subsidy_unicos}")

    # 5. Muestra de contenido de una columna JSON: 'employee_type' (primeros 5)
    print("\nMuestra de contenido de la columna JSON 'employee_type' (primeros 5):")
    cursor.execute("SELECT employee_type FROM employee_contracts WHERE employee_type IS NOT NULL LIMIT 5;")
    employee_type_sample = [fila[0] for fila in cursor.fetchall()]
    for i, etype in enumerate(employee_type_sample):
        print(f"  - Muestra {i+1}: {etype}")

except Exception as e:
    print(f"Ocurrió un error durante la verificación inicial de datos en 'employee_contracts': {e}")
finally:
    # Cerrar la conexión al finalizar el bloque para limpiar recursos
    if 'cursor' in locals() and cursor is not None:
        try:
            cursor.close()
        except Exception as e:
            print(f"Advertencia al cerrar cursor en finally: {e}")
    if 'conexion' in locals() and conexion is not None and conexion.is_connected():
        try:
            conexion.close()
        except Exception as e:
            print(f"Advertencia al cerrar conexión en finally: {e}")
    print("Conexión cerrada al finalizar el bloque.")


--- Bloque 8: Verificación Inicial de 'employee_contracts' ---
Reestableciendo conexión a la base de datos...
✅ Conexión exitosa a la base de datos
Conexión y cursor reestablecidos.

Total de registros en 'employee_contracts': 43

Valores nulos por columna en 'employee_contracts':
  - id: 0 nulos
  - employee_id: 0 nulos
  - employee_type: 0 nulos
  - employee_subtype: 0 nulos
  - high_penson_risk: 0 nulos
  - integral_salary: 0 nulos
  - type: 0 nulos
  - start_date: 0 nulos
  - end_date: 32 nulos
  - salary: 0 nulos
  - transport_subsidy: 0 nulos
  - risk_type: 0 nulos
  - holidays_enjoyed: 0 nulos
  - health_provider_id: 5 nulos
  - health_provider: 0 nulos
  - pension_provider: 2 nulos
  - pension_provider_id: 5 nulos
  - severance_provider: 2 nulos
  - severance_provider_id: 5 nulos
  - family_compensation_fund_provider: 0 nulos
  - family_compensation_fund_provider_id: 5 nulos
  - settlement_cause: 30 nulos
  - settlement_with_just_cause: 0 nulos
  - settlement_probationary_peri

#Resultado análisis y toma de acciones: 
health_provider_id: 5 nulos. Rellenaremos con N/A.
pension_provider: 2 nulos. Rellenaremos con N/A.
pension_provider_id: 2 nulos. Rellenaremos con N/A.
severance_provider: 2 nulos. Rellenaremos con N/A.
severance_provider_id: 2 nulos. Rellenaremos con N/A.
settlement_approved_at: 43 nulos. Esto significa que ningún contrato ha sido liquidado o aprobado para liquidación. Es importante mantener esto en cuenta para el análisis futuro (e.g., todos los contratos son activos o no han llegado a su fase de liquidación).
Aplanamiento de Columnas JSON
Limpieza y Transformación

In [ ]:
# Script para limpiar y aplanar 'employee_contracts' directamente
import pandas as pd
import json
import numpy as np # Asegúrate de que numpy esté importado
from db_connection import get_engine, get_connection 

print("\n--- Bloque 9: Limpieza y Transformación de 'employee_contracts' ---")

# --- PASO CRÍTICO: CERRAR Y REABRIR CONEXIÓN PARA ASEGURAR ESTADO LIMPIO ---
print("Reestableciendo conexión a la base de datos...")
conexion = None 
cursor = None 
try:
    if 'conexion' in globals() and conexion is not None and conexion.is_connected():
        if 'cursor' in globals() and cursor is not None:
            try:
                cursor.close()
            except Exception as e:
                print(f"Advertencia al cerrar cursor previo: {e}")
        conexion.close()
    
    conexion = get_connection()
    cursor = conexion.cursor()
    print("✅ Conexión y cursor reestablecidos.")
except Exception as e:
    print(f"❌ ERROR: No se pudo reestablecer la conexión a la base de datos: {e}")
    raise 

try:
    engine = get_engine()
    print("✅ Engine de base de datos obtenido.")

    # --- PASO 1: Cargando la tabla 'employee_contracts' en Pandas y realizando transformaciones ---
    print("\n--- PASO 1: Cargando 'employee_contracts' en Pandas y realizando transformaciones ---")
    df_contracts = pd.read_sql("SELECT * FROM employee_contracts;", engine)
    print(f"DataFrame de contratos cargado desde 'employee_contracts'. Registros: {len(df_contracts)}")

    # 1.1. Manejo de valores nulos y asegurando nombres de columnas válidos
    print("Rellenando valores nulos para columnas específicas con 'N/A' y asegurando nombres válidos...")
    cols_to_fill_na = [
        'health_provider_id', 
        'pension_provider',
        'pension_provider_id',
        'severance_provider',
        'severance_provider_id'
    ]
    for col in cols_to_fill_na:
        if col in df_contracts.columns:
            if 'id' not in col:
                df_contracts[col] = df_contracts[col].astype(str).replace('nan', 'N/A').fillna('N/A')
            else:
                df_contracts[col] = df_contracts[col].replace({np.nan: None})
        else:
            print(f"   Advertencia: Columna '{col}' no encontrada en el DataFrame para rellenar nulos.")
    
    df_contracts.columns = [str(col) if pd.notna(col) else f"unnamed_col_{i}" for i, col in enumerate(df_contracts.columns)]
    
    print("✅ Nombres de columnas del DataFrame validados.")
    print("✅ Relleno de nulos completado en DataFrame.")

    # 1.2. Aplanamiento de Columnas JSON
    print("\nIniciando aplanamiento de columnas JSON en Pandas...")
    json_columns = [
        'employee_type',
        'employee_subtype',
        'type',
        'risk_type',
        'health_provider',
        'pension_provider',
        'severance_provider',
        # 'family_compensation_fund_provider' <-- Sigue excluida como acordamos
    ]

    def extract_json_description_or_code(json_str):
        if pd.isna(json_str) or not isinstance(json_str, str) or not json_str.strip().startswith('{'):
            return None 
        try:
            data = json.loads(json_str)
            if isinstance(data, dict):
                if 'description' in data:
                    return str(data['description'])
                elif 'code' in data: 
                    return str(data['code'])
            return None 
        except json.JSONDecodeError:
            return None 
        except Exception as e:
            print(f"   Error inesperado al procesar JSON: {e} en '{str(json_str)[:50]}'")
            return None

    for col in json_columns:
        if col in df_contracts.columns:
            df_contracts[col] = df_contracts[col].astype(str)
            df_contracts[col] = df_contracts[col].apply(extract_json_description_or_code)
            print(f"   Columna '{col}' aplanada a 'description'/'code' o NULL.")
        else:
            print(f"   Advertencia: Columna '{col}' no encontrada en el DataFrame para aplanar JSON.")
    print("✅ Aplanamiento de JSONs en DataFrame completado.")
    
    # --- PASO 2: ELIMINADO - NO SE ALTERA LA ESTRUCTURA DE LA BASE DE DATOS ---

    # --- PASO 3: Actualizar los registros con los datos limpios del DataFrame ---
    print("\n--- PASO 3: Actualizando registros en 'employee_contracts' con datos limpios ---")
    
    cursor.execute("DESCRIBE `employee_contracts`;")
    db_columns_info = cursor.fetchall()
    db_column_names = [col_info[0] for col_info in db_columns_info]

    valid_columns_for_update = [
        col for col in df_contracts.columns
        if isinstance(col, str) and col != 'id' and col in db_column_names
    ]
    
    print(f"\nDEBUG: Columnas en el DataFrame: {df_contracts.columns.tolist()}")
    print(f"DEBUG: Columnas en la Base de Datos: {db_column_names}")
    print(f"DEBUG: Columnas que se usarán para UPDATE: {valid_columns_for_update}")

    if not valid_columns_for_update:
        print("⚠️ Advertencia: No hay columnas válidas para actualizar (aparte del 'id'). No se realizará ninguna actualización.")
    else:
        set_clauses = [f"`{col}` = %s" for col in valid_columns_for_update]
        update_sql = f"UPDATE `employee_contracts` SET {', '.join(set_clauses)} WHERE `id` = %s;"
        
        print(f"DEBUG: SQL de UPDATE generado: {update_sql[:150]}...")

        updated_count = 0
        for index, row in df_contracts.iterrows():
            update_values = []
            for col in valid_columns_for_update:
                val = row[col]
                # Conversión final de valores de Pandas/NumPy a tipos compatibles con MySQL
                # Corregido: Reemplazar np.float_ con np.float64 para compatibilidad con NumPy 2.0+
                if pd.isna(val) or (isinstance(val, (float, np.float64)) and np.isnan(val)):
                    update_values.append(None) 
                elif isinstance(val, (np.int64, np.int32)):
                    update_values.append(int(val)) 
                elif isinstance(val, (np.bool_)):
                    update_values.append(bool(val)) 
                else:
                    update_values.append(val)
                    
            update_values.append(row['id']) 
            
            try:
                cursor.execute(update_sql, tuple(update_values))
                updated_count += 1
                if updated_count % 10 == 0:
                    print(f"   Progreso: {updated_count}/{len(df_contracts)} registros procesados...")
            except Exception as e:
                print(f"❌ Error al actualizar el registro con ID {row['id']}: {e}")
                print(f"   Valores que intentaron insertarse para ID {row['id']}: {tuple(update_values)}")
                conexion.rollback()
                raise 

        conexion.commit() 
        print(f"✅ Actualización de {updated_count} registros en la tabla 'employee_contracts' completada exitosamente.")
    
    print("\n✅ Limpieza y transformación de 'employee_contracts' finalizada exitosamente.")

except Exception as e:
    print(f"❌ Ocurrió un error general durante la limpieza de 'employee_contracts': {e}")
    try:
        if 'conexion' in locals() and conexion is not None and conexion.is_connected():
            conexion.rollback()
            print("Transacción revertida debido a un error.")
    except Exception as rollback_e:
        print(f"❌ Error adicional al intentar revertir: {rollback_e}")
    
finally:
    print("Cerrando la conexión después del procesamiento.")
    try:
        if 'cursor' in locals() and cursor is not None:
            cursor.close()
        if 'conexion' in locals() and conexion is not None and conexion.is_connected():
            conexion.close()
        print("✅ Conexión cerrada exitosamente.")
    except Exception as e:
        print(f"❌ Advertencia al cerrar la conexión en finally: {e}")


--- Bloque 9: Limpieza y Transformación de 'employee_contracts' ---
Reestableciendo conexión a la base de datos...
✅ Conexión exitosa a la base de datos
✅ Conexión y cursor reestablecidos.
✅ Engine de base de datos obtenido.

--- PASO 1: Cargando 'employee_contracts' en Pandas y realizando transformaciones ---
DataFrame de contratos cargado desde 'employee_contracts'. Registros: 43
Rellenando valores nulos para columnas específicas con 'N/A' y asegurando nombres válidos...
✅ Nombres de columnas del DataFrame validados.
✅ Relleno de nulos completado en DataFrame.

Iniciando aplanamiento de columnas JSON en Pandas...
   Columna 'employee_type' aplanada a 'description'/'code' o NULL.
   Columna 'employee_subtype' aplanada a 'description'/'code' o NULL.
   Columna 'type' aplanada a 'description'/'code' o NULL.
   Columna 'risk_type' aplanada a 'description'/'code' o NULL.
   Columna 'health_provider' aplanada a 'description'/'code' o NULL.
   Columna 'pension_provider' aplanada a 'descrip

In [ ]:
# Cerrar conexión
cursor.close()
conexion.close()


In [34]:
import pandas as pd
import plotly.express as px

In [35]:
# Script para generar gráficos BI con Plotly Express (Distribución de Contratos por Tipo y Salario)

import pandas as pd
import json
import numpy as np
import plotly.express as px
from db_connection import get_engine, get_connection # Asegúrate de que este módulo esté accesible

print("\n--- Bloque 9: Generación de Gráfico BI - Distribución de Contratos por Tipo y Salario ---")

conexion = None
cursor = None
try:
    print("Estableciendo conexión a la base de datos para el aplanamiento y visualización...")
    conexion = get_connection()
    cursor = conexion.cursor()
    engine = get_engine()
    print("✅ Conexión y cursor reestablecidos exitosamente.")
    print("✅ Engine de base de datos obtenido.")

    # --- PASO CRÍTICO: Deshabilitar temporalmente las verificaciones de claves foráneas ---
    print("\nDeshabilitando temporalmente las verificaciones de claves foráneas...")
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")
    print("✅ Verificaciones de claves foráneas deshabilitadas.")

    # --- PASO 1: Modificar la estructura de la tabla 'employee_contracts' ---
    # Convertir la columna 'type' (y otras relevantes si fueran necesarias) a VARCHAR(255) NULL
    print("\nModificando la columna 'type' a VARCHAR(255) NULL en 'employee_contracts'...")
    alter_table_sql = """
        ALTER TABLE employee_contracts
        MODIFY COLUMN type VARCHAR(255) NULL;
    """
    try:
        cursor.execute(alter_table_sql)
        conexion.commit() # Confirmar el cambio de estructura
        print("✅ Columna 'type' modificada a VARCHAR(255) NULL exitosamente.")
    except Exception as e:
        conexion.rollback()
        print(f"❌ Error al modificar la columna 'type': {e}")
        raise # Re-lanza el error para detener el script

    # --- PASO CRÍTICO: Re-habilitar las verificaciones de claves foráneas ---
    print("\nRe-habilitando las verificaciones de claves foráneas...")
    cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")
    print("✅ Verificaciones de claves foráneas re-habilitadas.")

    # 2. Cargar la tabla 'employee_contracts' completa en un DataFrame de Pandas
    print("\nCargando la tabla 'employee_contracts' en un DataFrame de Pandas...")
    # Solo seleccionamos las columnas que necesitamos para el gráfico
    df_contracts = pd.read_sql("SELECT id, type, salary FROM employee_contracts;", engine)
    print(f"DataFrame de 'employee_contracts' cargado. Registros: {len(df_contracts)}")

    # Lista de columnas JSON a procesar (ahora estas columnas son VARCHAR en la DB)
    json_columns_to_flatten = ['type'] # Solo 'type' para este gráfico

    # 3. Función para extraer valores JSON
    def extract_json_value(json_str):
        if pd.isna(json_str) or not isinstance(json_str, str) or not json_str.strip():
            return None
        if not json_str.strip().startswith('{') and not json_str.strip().startswith('['):
            return json_str
        try:
            data = json.loads(json_str)
            if isinstance(data, dict):
                if 'description' in data and data['description'] is not None:
                    return str(data['description'])
                elif 'name' in data and data['name'] is not None:
                    return str(data['name'])
                elif 'code' in data and data['code'] is not None:
                    return str(data['code'])
            return None
        except json.JSONDecodeError:
            return json_str if json_str.strip() else None
        except Exception as e:
            return None

    # 4. Iterar sobre la columna JSON y aplanarla
    print("\nIniciando aplanamiento de la columna 'type'...")
    for col in json_columns_to_flatten:
        if col in df_contracts.columns:
            df_contracts[col] = df_contracts[col].astype(str)
            df_contracts[col] = df_contracts[col].apply(extract_json_value)
            print(f"   Columna '{col}' aplanada a valor extraído o NULL.")
        else:
            print(f"   Advertencia: Columna '{col}' no encontrada en el DataFrame.")
    print("✅ Aplanamiento de columnas completado.")

    # Renombrar la columna 'type' a algo más descriptivo para el gráfico
    df_contracts.rename(columns={'type': 'Contract_Type'}, inplace=True)

    # 5. Preparar los datos para el gráfico: Contar tipos de contrato y calcular salario promedio
    print("\nCalculando conteo y salario promedio por tipo de contrato...")
    contract_summary = df_contracts.groupby('Contract_Type').agg(
        contract_count=('id', 'count'),
        avg_salary=('salary', 'mean')
    ).reset_index()

    # Opcional: Reemplazar Nulos en avg_salary para visualización si es necesario
    contract_summary['avg_salary'] = contract_summary['avg_salary'].fillna(0).round(2)

    print("Resumen por tipo de contrato:")
    print(contract_summary)

    # 6. Generar el gráfico de barras interactivo con Plotly Express
    print("\nGenerando gráfico de barras interactivo de 'Contratos por Tipo y Salario Promedio'...")
    fig = px.bar(contract_summary,
                 x='Contract_Type',
                 y='contract_count',
                 title='Distribución de Contratos por Tipo y Salario Promedio',
                 labels={
                     'Contract_Type': 'Tipo de Contrato',
                     'contract_count': 'Número de Contratos'
                 },
                 hover_data={'avg_salary': ':.2f'}, # Muestra el salario promedio en el tooltip
                 text='contract_count', # Muestra el conteo en la barra
                 color='avg_salary', # Colorea las barras según el salario promedio
                 color_continuous_scale=px.colors.sequential.Plasma # Escala de color
                )

    fig.update_traces(texttemplate='%{text}', textposition='outside')
    fig.update_layout(xaxis_title="Tipo de Contrato",
                      yaxis_title="Número de Contratos",
                      title_x=0.5, # Centrar el título
                      hovermode="x unified"
                     )

    # 7. Mostrar el gráfico
    fig.show()
    print("✅ Gráfico 'Distribución de Contratos por Tipo y Salario Promedio' generado y mostrado.")

except Exception as e:
    print(f"\n❌❌ Ocurrió un error general: {e}")
    print("Por favor, revisa la conexión a la base de datos, la existencia de la tabla 'employee_contracts', y el contenido de la columna 'type'.")
    try:
        if conexion and conexion.is_connected():
            conexion.rollback()
            print("Transacción revertida debido a un error.")
    except Exception as rollback_e:
        print(f"❌ Error adicional al intentar revertir la transacción: {rollback_e}")
finally:
    if cursor is not None:
        try:
            cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")
            print("✅ Verificaciones de claves foráneas re-habilitadas en el bloque 'finally'.")
        except Exception as e:
            print(f"❌ Advertencia: No se pudieron re-habilitar las FKs en finally: {e}")
    if conexion and conexion.is_connected():
        conexion.close()
        print("✅ Conexión a la base de datos cerrada.")

print("\n--- Fin del Bloque de Generación de Gráficos ---")


--- Bloque 9: Generación de Gráfico BI - Distribución de Contratos por Tipo y Salario ---
Estableciendo conexión a la base de datos para el aplanamiento y visualización...
✅ Conexión exitosa a la base de datos
✅ Conexión y cursor reestablecidos exitosamente.
✅ Engine de base de datos obtenido.

Deshabilitando temporalmente las verificaciones de claves foráneas...
✅ Verificaciones de claves foráneas deshabilitadas.

Modificando la columna 'type' a VARCHAR(255) NULL en 'employee_contracts'...
✅ Columna 'type' modificada a VARCHAR(255) NULL exitosamente.

Re-habilitando las verificaciones de claves foráneas...
✅ Verificaciones de claves foráneas re-habilitadas.

Cargando la tabla 'employee_contracts' en un DataFrame de Pandas...
DataFrame de 'employee_contracts' cargado. Registros: 43

Iniciando aplanamiento de la columna 'type'...
   Columna 'type' aplanada a valor extraído o NULL.
✅ Aplanamiento de columnas completado.

Calculando conteo y salario promedio por tipo de contrato...
Resum

✅ Gráfico 'Distribución de Contratos por Tipo y Salario Promedio' generado y mostrado.
✅ Verificaciones de claves foráneas re-habilitadas en el bloque 'finally'.
✅ Conexión a la base de datos cerrada.

--- Fin del Bloque de Generación de Gráficos ---


In [38]:
# Script para generar gráficos BI con Plotly Express (Salario Promedio por Tipo de Contrato y Riesgo)

import pandas as pd
import json
import plotly.express as px
from db_connection import get_engine, get_connection # Asegúrate de que este módulo esté accesible

print("\n--- Bloque 11: Generación de Gráfico BI - Salario Promedio por Tipo de Contrato y Riesgo ---")

conexion = None
cursor = None
try:
    print("Estableciendo conexión a la base de datos para el aplanamiento y visualización...")
    conexion = get_connection()
    cursor = conexion.cursor()
    engine = get_engine()
    print("✅ Conexión y cursor reestablecidos exitosamente.")
    print("✅ Engine de base de datos obtenido.")

    # --- PASO CRÍTICO: Deshabilitar temporalmente las verificaciones de claves foráneas ---
    print("\nDeshabilitando temporalmente las verificaciones de claves foráneas...")
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")
    print("✅ Verificaciones de claves foráneas deshabilitadas.")

    # --- PASO 1: Modificar la estructura de la tabla 'employee_contracts' ---
    # Asegurarnos que 'type' y 'risk_type' son VARCHAR(255) NULL
    print("\nModificando columnas 'type' y 'risk_type' a VARCHAR(255) NULL en 'employee_contracts'...")
    alter_table_sql_type = """
        ALTER TABLE employee_contracts
        MODIFY COLUMN type VARCHAR(255) NULL;
    """
    alter_table_sql_risk_type = """
        ALTER TABLE employee_contracts
        MODIFY COLUMN risk_type VARCHAR(255) NULL;
    """
    try:
        cursor.execute(alter_table_sql_type)
        cursor.execute(alter_table_sql_risk_type)
        conexion.commit()
        print("✅ Columnas 'type' y 'risk_type' modificadas a VARCHAR(255) NULL exitosamente.")
    except Exception as e:
        conexion.rollback()
        print(f"❌ Error al modificar columnas: {e}")
        raise # Re-lanza el error para detener el script

    # --- PASO CRÍTICO: Re-habilitar las verificaciones de claves foráneas ---
    print("\nRe-habilitando las verificaciones de claves foráneas...")
    cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")
    print("✅ Verificaciones de claves foráneas re-habilitadas.")

    # 2. Cargar las columnas necesarias de 'employee_contracts'
    print("\nCargando datos de 'employee_contracts' (columnas type, risk_type, salary)...")
    df_contracts = pd.read_sql("SELECT type, risk_type, salary FROM employee_contracts;", engine)
    print(f"DataFrame de 'employee_contracts' cargado. Registros: {len(df_contracts)}")

    # 3. Función para extraer valores JSON
    def extract_json_value(json_str):
        if pd.isna(json_str) or not isinstance(json_str, str) or not json_str.strip():
            return None
        if not json_str.strip().startswith('{') and not json_str.strip().startswith('['):
            return json_str # Si no es JSON válido, pero es una cadena, la devuelve
        try:
            data = json.loads(json_str)
            if isinstance(data, dict):
                if 'description' in data and data['description'] is not None:
                    return str(data['description'])
                elif 'name' in data and data['name'] is not None:
                    return str(data['name'])
                elif 'code' in data and data['code'] is not None:
                    return str(data['code'])
            return None
        except json.JSONDecodeError:
            return json_str if json_str.strip() else None # Si no es JSON válido, pero es una cadena, la devuelve
        except Exception as e:
            return None

    # 4. Iterar sobre las columnas JSON y aplanarlas
    print("\nIniciando aplanamiento de columnas 'type' y 'risk_type'...")
    json_columns_to_flatten = ['type', 'risk_type']
    for col in json_columns_to_flatten:
        if col in df_contracts.columns:
            df_contracts[col] = df_contracts[col].astype(str)
            df_contracts[col] = df_contracts[col].apply(extract_json_value)
            print(f"   Columna '{col}' aplanada.")
        else:
            print(f"   Advertencia: Columna '{col}' no encontrada en el DataFrame.")
    print("✅ Aplanamiento de columnas completado.")

    # Renombrar columnas para claridad en el gráfico
    df_contracts.rename(columns={'type': 'Contract_Type', 'risk_type': 'Risk_Type'}, inplace=True)

    # 5. Calcular el salario promedio por Tipo de Contrato y Tipo de Riesgo
    print("\nCalculando salario promedio por Tipo de Contrato y Tipo de Riesgo...")
    # Asegúrate de no incluir valores de salario nulos en el cálculo
    df_contracts_cleaned = df_contracts.dropna(subset=['salary', 'Contract_Type', 'Risk_Type'])
    
    # Rellenar los Nulos en Contract_Type o Risk_Type con 'Desconocido' antes de agrupar, si es preferible
    df_contracts_cleaned['Contract_Type'] = df_contracts_cleaned['Contract_Type'].fillna('Desconocido')
    df_contracts_cleaned['Risk_Type'] = df_contracts_cleaned['Risk_Type'].fillna('Desconocido')


    avg_salary_by_type_risk = df_contracts_cleaned.groupby(['Contract_Type', 'Risk_Type'])['salary'].mean().reset_index()
    avg_salary_by_type_risk['salary'] = avg_salary_by_type_risk['salary'].round(2)

    print("Salario promedio por tipo de contrato y riesgo:")
    print(avg_salary_by_type_risk.head())

    # 6. Generar el gráfico de barras apiladas (o agrupadas) interactivo con Plotly Express
    print("\nGenerando gráfico de barras interactivo de 'Salario Promedio por Tipo de Contrato y Riesgo'...")
    fig = px.bar(avg_salary_by_type_risk,
                 x='Contract_Type',
                 y='salary',
                 color='Risk_Type', # Segmenta las barras por tipo de riesgo
                 title='Salario Promedio por Tipo de Contrato y Tipo de Riesgo Laboral',
                 labels={
                     'Contract_Type': 'Tipo de Contrato',
                     'salary': 'Salario Promedio',
                     'Risk_Type': 'Tipo de Riesgo'
                 },
                 barmode='group', # O 'stack' para barras apiladas
                 text_auto='.2s' # Muestra el valor de salario en las barras, formato corto
                )

    fig.update_layout(xaxis_title="Tipo de Contrato",
                      yaxis_title="Salario Promedio",
                      title_x=0.5, # Centrar el título
                      hovermode="x unified"
                     )

    # 7. Mostrar el gráfico
    fig.show()
    print("✅ Gráfico 'Salario Promedio por Tipo de Contrato y Riesgo' generado y mostrado.")

except Exception as e:
    print(f"\n❌❌ Ocurrió un error general: {e}")
    print("Por favor, revisa la conexión a la base de datos, la existencia de la tabla 'employee_contracts', y el contenido de las columnas 'type', 'risk_type', y 'salary'.")
    try:
        if conexion and conexion.is_connected():
            conexion.rollback()
            print("Transacción revertida debido a un error.")
    except Exception as rollback_e:
        print(f"❌ Error adicional al intentar revertir la transacción: {rollback_e}")
finally:
    if cursor is not None:
        try:
            cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")
            print("✅ Verificaciones de claves foráneas re-habilitadas en el bloque 'finally'.")
        except Exception as e:
            print(f"❌ Advertencia: No se pudieron re-habilitar las FKs en finally: {e}")
    if conexion and conexion.is_connected():
        conexion.close()
        print("✅ Conexión a la base de datos cerrada.")

print("\n--- Fin del Bloque de Generación de Gráficos ---")


--- Bloque 11: Generación de Gráfico BI - Salario Promedio por Tipo de Contrato y Riesgo ---
Estableciendo conexión a la base de datos para el aplanamiento y visualización...
✅ Conexión exitosa a la base de datos
✅ Conexión y cursor reestablecidos exitosamente.
✅ Engine de base de datos obtenido.

Deshabilitando temporalmente las verificaciones de claves foráneas...
✅ Verificaciones de claves foráneas deshabilitadas.

Modificando columnas 'type' y 'risk_type' a VARCHAR(255) NULL en 'employee_contracts'...
✅ Columnas 'type' y 'risk_type' modificadas a VARCHAR(255) NULL exitosamente.

Re-habilitando las verificaciones de claves foráneas...
✅ Verificaciones de claves foráneas re-habilitadas.

Cargando datos de 'employee_contracts' (columnas type, risk_type, salary)...
DataFrame de 'employee_contracts' cargado. Registros: 43

Iniciando aplanamiento de columnas 'type' y 'risk_type'...
   Columna 'type' aplanada.
   Columna 'risk_type' aplanada.
✅ Aplanamiento de columnas completado.

Calcu

✅ Gráfico 'Salario Promedio por Tipo de Contrato y Riesgo' generado y mostrado.
✅ Verificaciones de claves foráneas re-habilitadas en el bloque 'finally'.
✅ Conexión a la base de datos cerrada.

--- Fin del Bloque de Generación de Gráficos ---


In [40]:
import pandas as pd
from db_connection import get_engine, get_connection

conexion = None
try:
    engine = get_engine()
    conexion = get_connection()
    cursor = conexion.cursor()

    print("\nListando tablas en la base de datos 'novateam'...")
    cursor.execute("SHOW TABLES;")
    tables = cursor.fetchall()
    print("Tablas encontradas:")
    for table in tables:
        print(f"- {table[0]}") # table[0] contiene el nombre de la tabla
    print("✅ Listado de tablas completado.")

except Exception as e:
    print(f"\n❌❌ Ocurrió un error al listar tablas: {e}")
finally:
    if conexion and conexion.is_connected():
        conexion.close()

✅ Conexión exitosa a la base de datos

Listando tablas en la base de datos 'novateam'...
Tablas encontradas:
- accounting_account_balances
- accounting_accounts
- accounting_movements
- accounting_voucher_items
- accounting_voucher_types
- accounting_vouchers
- billing_numberings
- contact_relationships
- contact_statements
- contacts
- contract_salary_history
- costs_and_expenses
- costs_and_expenses_categories
- coupons
- document_items
- documents
- ecommerce_legal_info
- ecommerce_purchase_orders
- employee_contracts
- employee_positions
- employees
- fixed_asset_depreciations
- fixed_assets
- fixed_assets_groups
- headquarter_warehouses
- headquarters
- inventory_adjustments
- inventory_groups
- item_balance
- item_categories
- item_depreciations
- item_kardex
- item_subcategories
- item_variations
- items
- payment_conditions
- payments
- payroll_deductions
- payroll_details
- payroll_incomes
- payroll_providers
- payrolls
- price_lists
- retention_concepts
- retentions
- retenti